## <span style="color:#956bbf">Analysing Buyer Behaviour Using Consumer Panel Data</span>
---

### <span style="color:#956bbf">Introduction</span>

This is a python implementation of Bruce G.S. Hardie's [An Excel-based Introduction to Analysing Buyer Behaviour Using Consumer Panel Data](https://www.brucehardie.com/notes/042/) using [Polars](https://pola.rs).

Several leading market research firms (e.g., Kantar, IRI, Nielsen) collect so called consumer panel data and sell reports and analyses based on these data to interested parties. A large number of households are recruited to join the panel and they record all their grocery/HBA purchases. The idea of a panel is that we have repeated observations of the same people. The ability to track what individual households are purchasing over time can give important insights into what behavioural changes lie beneath observed changes in aggregate sales data. (terms panellist and household are used interchangeably) Nowadays, it is common for panellists to record their purchasing by scanning product barcodes via an app on their smartphones.

The objective of this notebook is to provide an introduction to basic analyses we can undertake using panel data. After briefing **describing the data** we will be working with, we present some preliminary **aggregate-level analyses**. Next we introduce some basic **brand performance measures** and consider the simple analyses that describe the **variation we observe in buyer behaviour** in a given time period. This analysis focuses on one brand at a time; next we consider some basic analyses that describe consumers’ **buying of multiple brands in a category**. We finish with some basic analyses that describe **how buyer behaviour evolves over time**, both for established products and new products.

Note: Whereas consumer panel data gives us information on the purchasing of a sample of customers for the whole category, the data in a firm’s transaction database gives us complete information on the purchasing of our products (but not those of our competitors). At a fundamental level, the types of reports developed by market research firms over the past 60+ years are a good starting point for the types of reports a firm should create as it seeks to understand the buying behaviour of its customers.

A traditional consumer panel works in the following manner.
1) When an individual first joins the panel, they complete a detailed questionnaire. A section of this questionnaire focuses on the demographics of their household. (This is typically updated once a year.)
2) After each shopping trip, each panellist records their purchases, scanning the barcode associated with each product and recording other information such as where the purchase was made (store or channel), the price paid, and the use of promotional deals. Twenty years ago, this would have been done using a custom handheld barcode scanner provided by the market research firm. These days, it is more common to use a smartphone app.
3) These data are uploaded to the research firm’s servers and merged with the purchases records of the other panellists. Each barcode is matched with detailed product information (e.g., category, brand, size, flavour) and this information is also stored in the database. The analyst can then create (typically product-category-specific) datasets for further analysis that tell us what each panellist purchased, when and where, and associated transaction- and/or product-specific data that may be of interest.
4) Panellists drop out of the panel all the time, and the research firm will be recruiting replacement households on a regular basis. When creating a dataset for further analysis, it is generally desirable to work with a so-called static sample of panellists, which comprises all those panellists active using the time period of interest; new panellists, as well as those that dropped out during the given time period, are excluded.

### <span style="color:#956bbf">Imports</span>
---

#### Import Packages

In [1]:
import polars as pl
import altair as alt
import numpy as np
from great_tables import GT, style, loc, md
import gc

# https://altair-viz.github.io/user_guide/display_frontends.html#displaying-in-jupyterlab
alt.renderers.enable('mimetype')
# alt.JupyterChart.enable_offline()
# alt.renderers.enable("jupyter", offline=True)

RendererRegistry.enable('mimetype')

#### Import Panel Data

We will make use of two datasets as we explore the basic types of summaries of buyer behaviour that can be created using consumer panel data. The first contains data on the purchasing of established brands in a mature product category, while the second contains data on the purchasing of a new product. Both datasets were created using static samples. While these are small datasets and contain a subset of the information available in the research firm’s databases, they are more than sufficient to convey the logic of creating the key summaries of buyer behaviour. Neither dataset includes data on the demographics of each panellist. As such, we will not consider how to create reports that explore how behaviours differ across demographic groups (e.g., by age or geography). However, anyone comfortable with the analyses undertaken in this note should be able to work out how to create such reports for themselves.

**Dataset 1**

The file `edible_grocery.csv` contains two years of data from a panel of 5021 households on their purchasing in an unnamed edible grocery product category. (We intentionally do not identity the category and the associated brand names.) There are 119 SKUs in this category. 91 SKUs are associated with the four largest brands in the category, which we have named Alpha, Bravo, Charlie, and Delta. The remaining SKUs belong to very low-share brands and we grouped them under the brand Other.

*Note*: SKU - A stock-keeping unit (SKU) is a unique combination of the attributes (e.g., brand, package type, package size, flavour) that define the products in the category.

Each record in this file consists of seven fields:
- `panel_id` A unique identifier for each household.
- `trans_id` A unique identifier for the purchase occasion.
- `week` The week number in which the purchase occurred. Week 1 corresponds to the calendar week starting on January 1, 20xy. Week 53 corresponds to the calendar week starting on December 31, 20xy.
- `sku_id` The SKU code.
- `units` The number of units purchased on the particular purchase occasion.
- `price` The price per unit paid at the point of purchase.
- `brand` The brand associated with the SKU purchased.

The associated file `sku_weight.csv` gives us the weight (in grams) of each SKU. There are two fields: `sku_id` and `weight`.

In [2]:
grocery_lf = pl.scan_csv(source="data/panel-datasets/edible_grocery.csv",
                         has_header=True,
                         separator=",",
                         schema={'panel_id': pl.UInt32,
                                 'trans_id': pl.Int32,
                                 'week': pl.UInt16,
                                 'sku_id': pl.UInt8,
                                 'units': pl.Int16,
                                 'price': pl.Float32,
                                 'brand': pl.Categorical})
grocery_lf.head().collect()

panel_id,trans_id,week,sku_id,units,price,brand
u32,i32,u16,u8,i16,f32,cat
3102011,1569,6,19,1,2.79,"""Alpha"""
3102012,4301,15,15,1,3.19,"""Alpha"""
3102012,4301,15,15,1,3.19,"""Alpha"""
3102012,4301,15,38,1,3.49,"""Bravo"""
3102012,4301,15,44,1,3.49,"""Bravo"""


In [3]:
sku_lf = pl.scan_csv(source="data/panel-datasets/sku_weight.csv",
                         has_header=True,
                         separator=",",
                         schema={'sku_id': pl.UInt8,
                                 'weight': pl.Int16})
sku_lf.head().collect()

sku_id,weight
u8,i16
1,400
2,400
3,400
4,250
5,1000


In [4]:
panel_size = 5021

**Dataset 2**

“Kiwi Bubbles” is a masked name for a shelf-stable juice drink, aimed primarily at children, which is sold as a multipack with several single-serve containers bundled together. Prior to national launch, it underwent a year-long test conducted in two of IRI’s BehaviorScan markets. The file `kiwibubbles_trans.csv` contains purchasing data for the new product, drawn from 1300 panellists in Market 1 and 1499 panellists in Market 2. (The purchasing of other brands in the category has been excluded from the dataset.)

Each record in this file consists of five fields:
- `ID` A unique identifier for each household.
- `Market` 1 or 2.
- `Week` The week in which the purchase occurred.
- `Day` The day of the week in which the purchase occurred. (The product was launched on day 1 of week 1.)
- `Units` The number of units of the new product
purchased on the particular purchase occasion.

In [5]:
kiwi_lf = pl.scan_csv(source="data/panel-datasets/kiwibubbles_trans.csv",
                      has_header=True,
                      separator=",",
                      schema={'ID': pl.UInt16,
                              'Market': pl.UInt8,
                              'Week': pl.Int16,
                              'Day': pl.Int16,
                              'Units': pl.Int16})
kiwi_lf.head().collect()

ID,Market,Week,Day,Units
u16,u8,i16,i16,i16
10001,1,19,3,1
10002,1,12,5,1
10003,1,37,7,1
10004,1,30,6,1
10004,1,47,3,1


#### Reusable Functions

In [6]:
# Weekly Grocery Sales LazyFrame (Query Plan): Weekly 'spend' by 'Category', 'Brand' or 'All'
def weekly_spend_summary(brand, lf):
    summary = (
        lf
        .select(['week', 'units', 'price', 'brand'])
        .with_columns(((pl.col('units') * pl.col('price'))).alias('spend'))
    )
    
    if brand == 'Category': # Return LazyFrame of total category
        summary = summary.group_by('week')
    elif brand == 'All': # Return LazyFrame of all brands
        summary = summary.group_by('week', 'brand')
    else:  # Return LazyFrame of specified brand
        summary = summary.filter(
            pl.col('brand') == brand
        ).group_by('week', 'brand')
        
    summary = summary.agg(
        pl.col("spend").sum().alias('Weekly Spend') 
    ).sort('week')
    
    return summary

In [7]:
# Weekly Grocery Volume Sales LazyFrame (Query Plan): Weekly 'volume' by 'Brand' or 'All'
def weekly_vol_summary(brand, lf):
    with pl.StringCache():
        lf = (
            lf
            .join(
                other=sku_lf,
                left_on="sku_id",
                right_on="sku_id"            
            )
            .select(['week', 'units', 'brand', 'weight'])
        )
        
        if brand != 'All': 
            brand = [brand] if type(brand) == str else brand
            lf = lf.filter(
                pl.col('brand').is_in(*[brand])
            )
            
        summary = lf.with_columns(
            (((pl.col('units') * pl.col('weight'))/1000)).alias('volume')
        ).group_by('week', 'brand').agg(
            pl.col("volume").sum().alias('Weekly Volume')
        ).sort('week')
        
    return summary

In [8]:
# Altair Weekly Line Plot
def weekly_plot(dataframe, y, year=2, color=None, title="", y_axis_label="", pct=False, legend=False):
    
    # Configure the color encoding only if color is provided
    if color is not None:
        color_encoding = alt.Color(
            f'{color}:N',  # N = a discrete unordered category
            legend=alt.Legend(title=color) if legend else None  # Add legend conditionally
        )
    else:
        color_encoding = alt.Color()  # No color encoding    
    
    chart = alt.Chart(dataframe).mark_line(strokeWidth=1).encode(
        x = alt.X(
            'week',
            axis=alt.Axis(
                values=np.arange(0, (year*52) + 1, 13), # Explicitly specify quarter-end weeks
                labelExpr="datum.value", # Show only these labels
                title='Week'
            )
        ),
        y = alt.Y(
            f'{y}:Q', # Q = a continuous real-valued quantity
            title=y_axis_label,
            axis=alt.Axis(format="$,.0f") if not pct else alt.Axis(format=",.0%")
        ),
        color = color_encoding
    ).properties(
        width=650,
        height=250,
        title=title
    ).configure_view(
        stroke=None
    ).configure_axisY(
        # grid=False # turn off y-axis grid if required
    )

    return chart # alt.JupyterChart(chart)

In [9]:
# Annual Sales Summary LazyFrame for All Brands
def annual_sales_summary():
    summary = (
        weekly_spend_summary('All', grocery_lf)
        .with_columns((pl.col("week") / 52).ceil().alias('year'))
        .group_by(['year', 'brand'])
        .agg(pl.col("Weekly Spend").sum().alias('Yearly Sales'))
    ).sort('year')
    
    return summary

In [10]:
def freq_dist_plot(
    data, 
    column, 
    bin_edges, 
    labels, 
    x_title, 
    y_title, 
    chart_title, 
    subtitle, 
    width=650, 
    height=250, 
    label_angle=0, 
    left_closed=True, 
    compute_rel_freq=True
):
    """
    Creates a standardized Altair bar chart for relative frequency distribution plots.

    Parameters:
    - data (Polars LazyFrame or DataFrame): Input dataset.
    - column (str): Column to analyze for distribution.
    - bin_edges (array-like): Edges for binning.
    - labels (list of str): Labels for the bins.
    - x_title (str): Title for the x-axis.
    - y_title (str): Title for the y-axis.
    - chart_title (str): Main title for the chart.
    - subtitle (str): Subtitle for the chart.
    - width (int, optional): Width of the chart. Default is 650.
    - height (int, optional): Height of the chart. Default is 250.
    - label_angle (int, optional): Angle for x-axis labels. Default is 0.
    - left_closed (bool, optional): Whether bins are left-closed. Default is True.
    - compute_rel_freq (bool, optional): Whether to compute relative frequencies. Default is True.

    Returns:
    - alt.Chart: The generated Altair chart.
    """
    # Apply binning to the data
    binned_data = data.with_columns(
        pl.col(column).cut(bin_edges, labels=labels, left_closed=left_closed).alias("cut")
    )

    # Optionally compute relative frequencies
    if compute_rel_freq:
        binned_data = (
            binned_data
            .group_by("cut")
            .agg(pl.col("cut").count().alias("Frequency"))
            .with_columns(
                (pl.col("Frequency") / pl.col("Frequency").sum()).alias("% of Total")
            )
            .collect()
        )

    # Create the Altair chart
    chart = alt.Chart(binned_data).mark_bar().encode(
        x=alt.X("cut:O", axis=alt.Axis(labelAngle=label_angle, title=x_title), sort=labels),
        y=alt.Y("% of Total:Q", axis=alt.Axis(format=".0%", title=y_title)),
    ).properties(
        width=width,
        height=height,
        title={"text": chart_title, "subtitle": subtitle},
    )

    return chart

### <span style="color:#956bbf">Preliminaries</span> 
---

Before we start analysing household-level behaviour, let us first get a sense of the general sales patterns observed in this category.

Our initial objective is to plot weekly revenue for all the brands and the overall category, respectively, and then plot weekly (volume) market shares for target brands.

#### Weekly Sales Pattern

In [11]:
# Weekly Sales Pivot Table - Polars DataFrame
# For visualizing and inspecting only
weekly_spend_summary('All', grocery_lf).collect().pivot(
    on="brand",
    index="week",
    values="Weekly Spend",
    sort_columns=True,
).with_columns(
    pl.sum_horizontal(pl.exclude('week')).alias("Total") # Row total
).sort("week")

week,Alpha,Bravo,Charlie,Delta,Other,Total
u16,f32,f32,f32,f32,f32,f32
1,331.460114,247.780121,51.790005,17.639999,18.75,667.420227
2,567.249939,398.640137,45.429996,34.190002,23.23,1068.740112
3,497.96994,639.980103,51.110001,39.389999,14.550001,1243.000122
4,1389.95752,472.5,51.869999,85.039993,39.779995,2039.147461
5,358.240204,252.540115,40.209995,69.540001,45.929993,766.460327
…,…,…,…,…,…,…
100,692.479919,420.479706,69.599991,54.590004,47.969997,1285.119629
101,265.770081,264.660065,64.029999,35.049999,26.880001,656.390198
102,379.610168,456.789429,43.499996,20.4,43.609993,943.909607


In [12]:
weekly_plot(dataframe=weekly_spend_summary('Category', grocery_lf).collect(), 
            y='Weekly Spend', 
            title='Category - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [13]:
weekly_plot(dataframe=weekly_spend_summary('Alpha', grocery_lf).collect(), 
            y='Weekly Spend', 
            title='Alpha - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [14]:
weekly_plot(dataframe=weekly_spend_summary('Bravo', grocery_lf).collect(), 
            y='Weekly Spend', 
            title='Bravo - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Weekly (Volume) Market Share

In [15]:
# Weekly (volume) market share
pct_volume =(
    weekly_vol_summary('All', grocery_lf)
    .collect()
    .pivot(
        on='brand',
        index='week',
        values='Weekly Volume',
        sort_columns=True
    )
    .with_columns(
        (pl.col("*").exclude("week")) / pl.sum_horizontal(pl.exclude('week'))
    )
)
pct_volume

week,Alpha,Bravo,Charlie,Delta,Other
u16,f64,f64,f64,f64,f64
1,0.481077,0.355011,0.11194,0.02532,0.026652
2,0.524953,0.371525,0.059391,0.029696,0.014435
3,0.389338,0.508812,0.060676,0.032505,0.008668
4,0.677032,0.230511,0.036686,0.044984,0.010787
5,0.428735,0.324306,0.07574,0.119348,0.051871
…,…,…,…,…,…
100,0.545707,0.31711,0.080381,0.039087,0.017715
101,0.36891,0.40552,0.140805,0.059138,0.025627
102,0.342581,0.543494,0.071147,0.019689,0.023089


In [16]:
with pl.StringCache():
    pct_volume_plot = (
        weekly_vol_summary('All', grocery_lf)
        .group_by('week')
        .agg(
            pl.col('Weekly Volume').sum().alias('Total Weekly Volume')
        )
        .join(
            other=weekly_vol_summary(['Alpha', 'Bravo'], grocery_lf),
            on='week',
        )
        .filter(
            (pl.col('brand').is_in(['Alpha', 'Bravo']))
        )
        .with_columns(
            # compute brand wise % of total volume sale
            (pl.col('Weekly Volume') / pl.col('Total Weekly Volume')).alias('pct_volume')
        )
    ).collect()

weekly_plot(dataframe=pct_volume_plot, 
            y='pct_volume', 
            color='brand', 
            title="Volume Market Share - Alpha vs. Bravo", 
            y_axis_label="% of Weekly Total", 
            pct=True, 
            legend=True)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


There appears to be a **high level of competition** between these two brands. What is the correlation in their market shares?

We observe that there is a strong negative correlation between the shares of Alpha and Bravo: an increase in one brand’s share is associated with a corresponding decrease in the share of the other brand.

In [17]:
corr_matrix = (
    pct_volume.select(
        pl.col('*').exclude('week')  # Exclude 'week' column
    )
    .corr()  # Compute the correlation matrix
    .with_columns(
        pl.Series(pct_volume.columns[1:]).alias("index")
    )
)

(
    GT(corr_matrix, rowname_col='index')
    .tab_header(title="Correlation Matrix of Weekly (Volume) Market Share")
    .fmt_number(columns=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'])
    .data_color(
        domain=[-1, 1],
        palette=["rebeccapurple", "white", 'orange']
    )
    .tab_style(
        style=[
            style.text(color="black"),
            style.fill('white'),
            
        ],
        locations=[
            loc.body(0,0), 
            loc.body(1,1),
            loc.body(2,2),
            loc.body(3,3),
            loc.body(4,4)
        ]
    )    
)

GT(_tbl_data=shape: (5, 6)
┌───────────┬───────────┬───────────┬───────────┬───────────┬─────────┐
│ Alpha     ┆ Bravo     ┆ Charlie   ┆ Delta     ┆ Other     ┆ index   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---     │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ str     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════╡
│ 1.0       ┆ -0.853061 ┆ -0.34158  ┆ -0.109693 ┆ -0.148193 ┆ Alpha   │
│ -0.853061 ┆ 1.0       ┆ -0.114179 ┆ -0.231641 ┆ -0.104983 ┆ Bravo   │
│ -0.34158  ┆ -0.114179 ┆ 1.0       ┆ 0.131988  ┆ 0.243434  ┆ Charlie │
│ -0.109693 ┆ -0.231641 ┆ 0.131988  ┆ 1.0       ┆ 0.308354  ┆ Delta   │
│ -0.148193 ┆ -0.104983 ┆ 0.243434  ┆ 0.308354  ┆ 1.0       ┆ Other   │
└───────────┴───────────┴───────────┴───────────┴───────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x10f74fcb0>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10f74f770>, _spanners=Spanners([]), _heading=Heading(title='Correlation Matrix of Weekly (Volume) Market Share', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffa500')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#cbb9dc')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#eee9f4')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e8e1f0')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=Non

#### Annual Sales

Let us finish this preliminary analysis of the data by computing the annual sales of each brand.

In [18]:
annual_sales_pivot = annual_sales_summary().collect().pivot(
    on='brand',
    index='year',
    sort_columns=True
).with_columns(
    pl.sum_horizontal(pl.all().exclude('year')).alias("Total") # add totals column
)

(
    GT(annual_sales_pivot, rowname_col="year")
    .tab_header(title="Annual Sales ($)")
    .tab_stubhead(label="Year")
    .fmt_currency()
    .fmt_integer(columns='year')
    .data_color(
        columns=annual_sales_pivot.columns[:-1],
        domain=[100, 36_000],
        palette=["white", "rebeccapurple"]
    )    
)

GT(_tbl_data=shape: (2, 7)
┌──────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬──────────────┐
│ year ┆ Alpha        ┆ Bravo        ┆ Charlie     ┆ Delta       ┆ Other       ┆ Total        │
│ ---  ┆ ---          ┆ ---          ┆ ---         ┆ ---         ┆ ---         ┆ ---          │
│ f64  ┆ f32          ┆ f32          ┆ f32         ┆ f32         ┆ f32         ┆ f32          │
╞══════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╪══════════════╡
│ 1.0  ┆ 33570.949219 ┆ 28603.345703 ┆ 5120.869629 ┆ 3271.509766 ┆ 1535.22998  ┆ 72101.90625  │
│ 2.0  ┆ 35250.738281 ┆ 26926.847656 ┆ 3922.679688 ┆ 2820.810303 ┆ 1739.820068 ┆ 70660.898438 │
└──────┴──────────────┴──────────────┴─────────────┴─────────────┴─────────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x10f7d7100>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.stub: 2>, column_label='year', column_align='right', column_width=None), ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa24cd0>, _spanners=Spanners([]), _heading=Heading(title='Annual Sales ($)', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7041a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69379b')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865dae')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8d67b3')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[0]), grpname=None, colname='Charlie', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None,

In [19]:
annual_change_sales = annual_sales_pivot.with_columns(
    pl.col('*').exclude('year').pct_change()
).filter(
    pl.col('year') == 2
).unpivot(
    index="year",  # Keep 'year' as a fixed identifier
    value_name='% Change',  
    variable_name='brand'
)

(
    GT(annual_change_sales, rowname_col='brand')
    .tab_header(title='% Change in Annual Sales')
    .tab_stubhead(label="Brands")
    .fmt_percent()
    .data_color(
        domain=[-0.3, 0.3],
        palette=['orange', 'white', 'rebeccapurple']
    ) 
    .cols_hide('year')   
    .cols_label(brand='Y-o-Y % Change')
)

GT(_tbl_data=shape: (6, 3)
┌──────┬─────────┬───────────┐
│ year ┆ brand   ┆ % Change  │
│ ---  ┆ ---     ┆ ---       │
│ f64  ┆ str     ┆ f32       │
╞══════╪═════════╪═══════════╡
│ 2.0  ┆ Alpha   ┆ 0.050037  │
│ 2.0  ┆ Bravo   ┆ -0.058612 │
│ 2.0  ┆ Charlie ┆ -0.233982 │
│ 2.0  ┆ Delta   ┆ -0.137765 │
│ 2.0  ┆ Other   ┆ 0.133263  │
│ 2.0  ┆ Total   ┆ -0.019986 │
└──────┴─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x10f78f570>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.hidden: 4>, column_label='year', column_align='right', column_width=None), ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='Y-o-Y % Change', column_align='left', column_width=None), ColInfo(var='% Change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa26210>, _spanners=Spanners([]), _heading=Heading(title='% Change in Annual Sales', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[2]), grpname=None, colname='year', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[3]), grpname=None, colname='year', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[4]), grpname=None, colname='year', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[5]), grpname=None, colname='year', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', r

In [20]:
chart = alt.Chart(annual_sales_summary().collect()).mark_bar().encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(title='Brands', labelAngle=0)),
    xOffset="year:N",
    y=alt.Y("Yearly Sales:Q",
            axis=alt.Axis(format="$,.0f"),
            title='Yearly Revenue ($)'),
    color=alt.Color("year:N", title='Year'),
).properties(
    width=650,
    height=250,
    title='Year 1 & Year 2 Revenues'
).show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [21]:
base = alt.Chart(annual_change_sales).encode(
    x=alt.X("brand:N", 
            title='Brand',
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y("% Change:Q",
            axis=alt.Axis(format=".0%")
    ),
    color=alt.condition(
        alt.datum["% Change"] > 0, # fixed this 
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    ),
    text=alt.Text(
        '% Change',
        format=(".1%")
    )
        
).properties(
    width=650,
    height=250,
    title='Y-o-Y % Change in Revenue'
)

chart = base.mark_bar() + base.mark_text(align='center', 
                                 baseline=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, 'bottom', 'top')),
                                 dy=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, -2, 2)),
                                 dx=0)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Annual Market Share

In [22]:
market_share = annual_sales_pivot.with_columns(
    pl.col('*').exclude('year') / pl.col('Total')
)

(
    GT(market_share, rowname_col="year")
    .tab_header(title="Annual Market Share (%)")
    .tab_stubhead(label="Year")
    .fmt_percent()
    .cols_hide('Total')
    .data_color(
        domain=[0, 0.5],
        palette=["white", "rebeccapurple"]
    )    
)

GT(_tbl_data=shape: (2, 7)
┌──────┬──────────┬──────────┬──────────┬──────────┬──────────┬───────┐
│ year ┆ Alpha    ┆ Bravo    ┆ Charlie  ┆ Delta    ┆ Other    ┆ Total │
│ ---  ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---   │
│ f64  ┆ f32      ┆ f32      ┆ f32      ┆ f32      ┆ f32      ┆ f32   │
╞══════╪══════════╪══════════╪══════════╪══════════╪══════════╪═══════╡
│ 1.0  ┆ 0.465604 ┆ 0.396707 ┆ 0.071023 ┆ 0.045373 ┆ 0.021293 ┆ 1.0   │
│ 2.0  ┆ 0.498872 ┆ 0.381071 ┆ 0.055514 ┆ 0.03992  ┆ 0.024622 ┆ 1.0   │
└──────┴──────────┴──────────┴──────────┴──────────┴──────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x11aa54e50>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.stub: 2>, column_label='year', column_align='right', column_width=None), ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.hidden: 4>, column_label='Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x108836d70>, _spanners=Spanners([]), _heading=Heading(title='Annual Market Share (%)', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7141a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865dae')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8a64b1')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[0]), grpname=None, colname='Charlie', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e9e2f1')]), StyleInfo(locname=LocBody(columns='Ch

In [23]:
annual_change_share = market_share.drop('Total').with_columns(
    pl.col('*').exclude('year').pct_change()
).filter(
    pl.col('year') == 2
).unpivot(
    index="year",  # Keep 'year' as a fixed identifier
    value_name='% Change',  
    variable_name='brand'
)

(
    GT(annual_change_share, rowname_col='brand')
    .tab_header(title='% Change in Market Share')
    .tab_stubhead(label="Brands")
    .fmt_percent()
    .data_color(
        domain=[-0.3, 0.3],
        palette=['orange', 'white', 'rebeccapurple']
    ) 
    .cols_hide('year')   
    .cols_label(brand='Y-o-Y % Change')
)

GT(_tbl_data=shape: (5, 3)
┌──────┬─────────┬───────────┐
│ year ┆ brand   ┆ % Change  │
│ ---  ┆ ---     ┆ ---       │
│ f64  ┆ str     ┆ f32       │
╞══════╪═════════╪═══════════╡
│ 2.0  ┆ Alpha   ┆ 0.071451  │
│ 2.0  ┆ Bravo   ┆ -0.039414 │
│ 2.0  ┆ Charlie ┆ -0.21836  │
│ 2.0  ┆ Delta   ┆ -0.120181 │
│ 2.0  ┆ Other   ┆ 0.156374  │
└──────┴─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x11aa65010>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.hidden: 4>, column_label='year', column_align='right', column_width=None), ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='Y-o-Y % Change', column_align='left', column_width=None), ColInfo(var='% Change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aab48a0>, _spanners=Spanners([]), _heading=Heading(title='% Change in Market Share', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[2]), grpname=None, colname='year', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[3]), grpname=None, colname='year', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[4]), grpname=None, colname='year', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=

In [24]:
base = alt.Chart(annual_change_share).encode(
    x=alt.X("brand:N",
            title='Brands', 
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y("% Change:Q",
            axis=alt.Axis(format=".0%")
    ),
    color=alt.condition(
        alt.datum["% Change"] > 0, # fixed this 
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    ),
    text=alt.Text(
        '% Change',
        format=(".1%")
    )
        
).properties(
    width=650,
    height=250,
    title='% Change in Annual Market Share'
)

chart = base.mark_bar() + base.mark_text(align='center', 
                                 baseline=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, 'bottom', 'top')),
                                 dy=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, -3, 3)),
                                 dx=0)

chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Average SKU Price

In [25]:
avg_sku_price = grocery_lf.select(
    ['week', 'sku_id', 'price', 'brand']
).filter(
    (pl.col('week') <= 52) & # pricing in the first year
    (pl.col('brand') == 'Alpha') # Filter by Alpha brand 
).group_by('brand', 'sku_id').agg(
    pl.col('price').mean()
).drop('brand').sort(
    pl.col('sku_id').cast(pl.Int8)
)

(
    GT(avg_sku_price.collect(), rowname_col='sku_id') 
    .tab_header(title='Alpha - Average SKU Retail Prices')
    .tab_stubhead(label="SKU IDs")
    .fmt_currency()
    .data_color(
        domain=[1, 15],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(price='Average Price ($)')
)

GT(_tbl_data=shape: (18, 2)
┌────────┬──────────┐
│ sku_id ┆ price    │
│ ---    ┆ ---      │
│ u8     ┆ f64      │
╞════════╪══════════╡
│ 4      ┆ 1.84705  │
│ 5      ┆ 3.438095 │
│ 6      ┆ 2.613991 │
│ 7      ┆ 5.099062 │
│ 8      ┆ 12.43    │
│ …      ┆ …        │
│ 18     ┆ 3.274    │
│ 19     ┆ 2.620052 │
│ 20     ┆ 3.43361  │
│ 21     ┆ 2.99     │
│ 22     ┆ 2.09     │
└────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x10f67ac90>, _boxhead=Boxhead([ColInfo(var='sku_id', type=<ColInfoTypeEnum.stub: 2>, column_label='sku_id', column_align='center', column_width=None), ColInfo(var='price', type=<ColInfoTypeEnum.default: 1>, column_label='Average Price ($)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10f7939b0>, _spanners=Spanners([]), _heading=Heading(title='Alpha - Average SKU Retail Prices', subtitle=None, preheader=None), _stubhead='SKU IDs', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='sku_id', rows=[0]), grpname=None, colname='sku_id', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ded3e9')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[1]), grpname=None, colname='sku_id', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d3c5e2')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[2]), grpname=None, colname='sku_id', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c8b6db')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[3]), grpname=None, colname='sku_id', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#bda8d3')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[4]), grpname=None, colname='sku_id', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b299cc')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[5]), grpname=None, colname='sku_id', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#a88ac5')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[6]), grpname=None, colname='sku_id', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9d7cbd')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[7]), grpname=None, colname='sku_id', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#926db6')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[8]), grpname=None, colname='sku_id', rownum=8, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#875faf')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[9]), grpname=None, colname='sku_id', rownum=9, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None,

In [26]:
avg_sku_price.drop(pl.col('sku_id')).describe()

statistic,price
str,f64
"""count""",18.0
"""null_count""",0.0
"""mean""",4.042794
"""std""",2.947076
"""min""",1.84705
"""25%""",2.582056
"""50%""",2.99
"""75%""",3.438095
"""max""",12.43


#### Garbage Collect

In [27]:
exceptions = ['grocery_lf', 'sku_lf', 'kiwi_lf', 'In', 'exceptions', 'active_variables']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (pl.LazyFrame, pl.DataFrame, pl.Series, alt.Chart, alt.LayerChart, list, int, float, str, np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var

gc.collect()

433

### <span style="color:#956bbf">Exploring Variation in Buyer Behaviour</span>
---

#### Introduction

The objective here is to explore how to perform the basic analyses that describe the variation we observe in buyer behaviour in a given time period. We will continue to work with the edible grocery dataset, exploring both purchase frequency and spend.

Before we can doing any analysis, we need to create some summary datasets. The first will summarise **how many times each panellist purchased each brand as well as in the category**. The second will summarise **how much each panellist spent on each brand and in the category**.

In [28]:
grocery_lf.filter(
    (pl.col('panel_id') == 3102016) &
    (pl.col('trans_id') == 844)
).collect()

panel_id,trans_id,week,sku_id,units,price,brand
u32,i32,u16,u8,i16,f32,cat
3102016,844,4,5,1,3.49,"""Alpha"""
3102016,844,4,5,1,3.49,"""Alpha"""
3102016,844,4,5,1,3.49,"""Alpha"""
3102016,844,4,15,1,3.49,"""Alpha"""
3102016,844,4,15,1,3.49,"""Alpha"""
3102016,844,4,89,1,2.49,"""Delta"""


On this one shopping trip (`trans_id` = 844), panellist `3102021` purchased a total of six items: three packs of SKU 5, two packs of SKU 15, and one pack of SKU 89. They purchased two different brand, Alpha and Delta.

Why are three lines for SKU 5 and not one line with units = 3. This is simply a function of how the items were scanned at the checkout. Some checkout operators will scan the three items separately; this would result in three lines in the transaction file, each with units = 1. Others will scan the item and press “3” on their till, resulting in one line in the transaction file with units = 3.

By convention, this purchase occasion is recorded as one **category transaction**, one Alpha transaction, and one Delta transaction. The number of units of Alpha purchased is five. When we say that the panellist made one category transaction, we mean they purchased at least one item in the category on that shopping trip. When we say that the panellist made one Alpha transaction, we mean they purchased at least one item associated with the brand on that shopping trip.

In order to analyse buyer behaviour in terms of transactions, we need to know the **number of brand and category transactions for each person**. We cannot work directly with the dataset we have been using; we effectively need to collapse the five rows associated with Alpha into one. This will require some intermediate analysis, which we undertake in the following manner.

Result of the intermediate prep should produce a dataset that summarizes transaction 844 by panellist 3102016 as 1 occasion of Alpha brand purchase and 1 occasion of Delta brand purchase:
<div>
<img src="references\intermediate-data-prep.png" width="1000"/>
</div>

#### Panellist-Level Data Preparation

<div style="max-width:600px;margin-left: auto; margin-right: auto;">
<img src="references\Consumer-Panel-Data-Relationship.png" width="600"/>
</div>

**One row per transaction**, with a binary indicator of whether or not each brand was purchased.

- Filter for year == 1, group by transaction ID, panel ID and brand, then count unique panel_ids per transaction and brand
- Each panelist has done some number of transactions. Group unique transactions and obtain the count of transactions occasions in which any listed brand was purchased
- Note: this is not a sum of total units (by brand) purchased per panellist.
- Note: Because there are multiple entires for the same transaction ID, we are not doing a count of brand instance per panelist (without trans_id grouping) as this would duplicate the occasions in some cases.

- Produce a dataset where there is just one row per transaction, with a binary indicator of whether or not each brand was purchased.
- Create a new column to track category purchase. Each row is an instance of at least one brand purchase so category column is set to 1.

**Panellist-level *transaction* summary**

The function below executes a query plan which summarizes each panellist's purchase occasions (transactions) for each brand

In [29]:
def trans_summary(brand, lf, year):
    
    # Primary Step: Filter by Year 1 and Remove Unused Columns
    filtered_lf = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') > ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    )

    # Intermediate Step: Group by trans_id, panel_id, and brand
    group_trans = filtered_lf.drop(
        pl.col('price', 'units')
    ).group_by(
        'trans_id', 'panel_id', 'brand'
    ).n_unique()
    
    if brand == "Category":
        # Panellist-level category transaction summary
        summary = group_trans.group_by(
            'panel_id'
        ).n_unique()
    else:
        # Panellist-level brand transaction summary
        summary = group_trans.filter(
            pl.col('brand') == brand
        ).group_by(
            'panel_id'
        ).n_unique()
    
    return summary.select(
        pl.col('panel_id'),
        pl.col('trans_id').alias('# of Purchases'),
        pl.col('brand').alias('Brands Purchased')
    )

In [30]:
def trans_pivot(lf, year):
    
    # Primary Step: Filter by Year 1 and Remove Unused Columns
    filtered_lf = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') > ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    )

    # Intermediate Step: Group by trans_id, panel_id, and brand
    group_trans = filtered_lf.drop(
        pl.col('price', 'units')
    ).group_by(
        'trans_id', 'panel_id', 'brand'
    ).n_unique() # count of unique entires
    
    summary = group_trans.collect().pivot(
        on='brand',
        index='panel_id',
        values='panel_id',
        aggregate_function="len"
        
    ).join(
        other=group_trans.group_by('panel_id').n_unique().drop('brand').collect(),
        on='panel_id'
    ).rename(
        {'trans_id': 'Category'}
    ).drop(
        pl.col('panel_id')
    )
    
    return summary

**Panellist-level *spend* summary**

The function below executes a query plan which summarizes the amount each panellist spent on each brand (and in the category) during year 1.

In [31]:
def spend_summary(brand, lf, year):
    
    group_spend = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') > ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    ).with_columns(
        ((pl.col('units') * pl.col('price'))).alias('spend')
    )
    
    if brand == "Category":
        # Panellist-level category spend summary
        summary = group_spend.drop(
            pl.col('units', 'price', 'brand')
        ).group_by(
            'panel_id'
        ).agg(
            pl.col('spend').sum()
        )
    else:
        # Panellist-level brand spend summary
        summary = group_spend.drop(
            pl.col('units', 'price')
        ).group_by(
            'panel_id', 'brand'
        ).agg(
            pl.col('spend').sum()
        ).filter(
            pl.col('brand') == brand
        ).drop('brand')
        
    return summary

In [32]:
def spend_pivot(lf, year):
    
    group_spend = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') > ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    ).with_columns(
        ((pl.col('units') * pl.col('price'))).alias('spend')
    ).collect().pivot(
        on='brand',
        index='panel_id',
        values='spend',
        aggregate_function='sum'
    ).select(
        'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'
    )

    return group_spend

**Panellist-level *volume purchasing* summary**

The function below executes a query plan which summarises each panellist’s volume purchasing in year 1.

In [33]:
def vol_summary(brand, lf, year):
    with pl.StringCache():
        group_vol = lf.filter(
            (pl.col('week') <= (year * 52)) &
            (pl.col('week') > ((year - 1) * 52))
        ).join(
            other=sku_lf,
            left_on='sku_id',
            right_on='sku_id'
        ).drop(
            pl.col('week','sku_id')
        ).with_columns(
            # volume column that is the product of weight of each SKU and the units of SKU sold
            (((pl.col('units') * pl.col('weight'))/1000)).alias('volume') # # weight from grams to kilograms
        ).drop(
            pl.col('units', 'price', 'weight')
        )
        
        if brand == "Category":
            # Panellist-level category volume sales summary
            summary = group_vol.drop(
                pl.col('brand')
            ).group_by(
                'panel_id'
            ).agg(
                pl.col('volume').sum()
            )
        else:
            # Panellist-level brand volume sales summary
            summary = group_vol.group_by(
                'panel_id', 'brand'
            ).agg(
                pl.col('volume').sum()
            ).filter(
                pl.col('brand') == brand
            ).drop('brand')
        
    return summary

In [34]:
def vol_pivot(lf, year):
    with pl.StringCache():
        group_vol = lf.filter(
            (pl.col('week') <= (year * 52)) &
            (pl.col('week') > ((year - 1) * 52))
        ).join(
            other=sku_lf,
            left_on='sku_id',
            right_on='sku_id'
        ).drop(
            pl.col('week','sku_id')
        ).with_columns(
            # volume column that is the product of weight of each SKU and the units of SKU sold
            (((pl.col('units') * pl.col('weight'))/1000)).alias('volume') # # weight from grams to kilograms
        ).drop(
            pl.col('units', 'price', 'weight')
        ).collect().pivot(
            on='brand',
            index='panel_id',
            values='volume',
            aggregate_function='sum'
        ).select(
            'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'
        )
        
    return group_vol

Looking at panellist-level spend and panellist-level volume purchasing, we note that for each row, `Category` equals the sum of the brand numbers, as we would expect. But this is not always the case in panellist-level transactions. Why is the sum of the brand-specific numbers sometimes greater than the associated category number?

Answer: Because a single transactions (once grouped) can have occasions of multiple brand purchases but the transaction only counts as one occasion of a category purchase. Consider the panellist `3102016` and her transaction ID `844`, The 4 entires on the one transaction ID count as a single category transaction, a single Alpha brand transaction and a single Delta transaction. The category transaction count is not a sum of brand purchase transaction count because regardless of the brand(s) and quantities purchased, the panelist purchased the category once in that transaction only.

#### Examining Purchase Frequency

Two standard brand performance metrics that summarize purchasing behaviour are **penetration** and **purchases per buyer (PPB)**. 

- *Penetration* is the percentage of households buying the product/category at least once in the given time period. 
  - In order to compute this, we need to know the number of panellists buying the product at least once in the time period of interest and the size of the panel during this period.
- *Purchases per buyer (PPB)* is the average number of times (separate shopping trips) the product/category was purchased (in the given time period) by those households that made at least one product/category purchase (in the given time period).
  - i.e. purchases per buyer (PPB) is the average number of times the product was purchased (in the given time period) per buyer. 
  - This is computed as the total number of purchase occasions on which the product was purchased by the panellists (in the time period of interest) divided by the number of panellists that purchased the product at least once (in the time period of interest).

Looking at *panellist-level transaction summary*, we see that there are 4574 rows in this table, meaning that we have summary data on the purchasing of 4574 households in year 1. But there are 5021 households in the panel. What has happened to the remaining 447 households? They did not make any category purchase during the year. (But they will have purchased in other categories.)

In [35]:
# the number of panellists who purchased each brand at least once in year 1
buyers = trans_pivot(grocery_lf, 1).count().select(
    'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other', 'Category'
).unpivot(
    variable_name='brand',
    value_name='buyers'
)

(
    GT(buyers, rowname_col='brand')
    .tab_header(title='Number of Buyers in Year 1')
    .tab_stubhead(label="Brands/Category")
    .fmt_integer()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[100, 3_000],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(buyers='# Of Buyers')
)

GT(_tbl_data=shape: (6, 2)
┌──────────┬────────┐
│ brand    ┆ buyers │
│ ---      ┆ ---    │
│ str      ┆ u32    │
╞══════════╪════════╡
│ Alpha    ┆ 2624   │
│ Bravo    ┆ 2562   │
│ Charlie  ┆ 813    │
│ Delta    ┆ 380    │
│ Other    ┆ 176    │
│ Category ┆ 4574   │
└──────────┴────────┘, _body=<great_tables._gt_data.Body object at 0x11aa4f5f0>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='buyers', type=<ColInfoTypeEnum.default: 1>, column_label='# Of Buyers', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x105c0c380>, _spanners=Spanners([]), _heading=Heading(title='Number of Buyers in Year 1', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[0]), grpname=None, colname='buyers', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7a4da6')]), StyleInfo(locname=LocBody(columns='buyers', rows=[1]), grpname=None, colname='buyers', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7d52a8')]), StyleInfo(locname=LocBody(columns='buyers', rows=[2]), grpname=None, colname='buyers', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cde6')]), StyleInfo(locname=LocBody(columns='buyers', rows=[3]), grpname=None, colname='buyers', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f0ebf5')]), StyleInfo(locname=LocBody(columns='buyers', rows=[4]), grpname=None, colname='buyers', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), Ce

In [36]:
# On how many purchase occasions did the brand/category buyers of each brand/category buy that brand/category?
# i.e, On how many purchase occasions did the 2624 buyers of Alpha buy Alpha?
transactions = trans_pivot(grocery_lf, 1).sum().select(
    'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other', 'Category'
).unpivot(
    variable_name='brand',
    value_name='transactions'
)

(
    GT(transactions, rowname_col='brand')
    .tab_header(title='Purchase Occasions by Buyers')
    .tab_stubhead(label="Brands/Category")
    .fmt_integer()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[400, 10_000],
        palette=[ 'white', 'rebeccapurple']
    )  
    .cols_label(transactions='# Of Transactions')
)

GT(_tbl_data=shape: (6, 2)
┌──────────┬──────────────┐
│ brand    ┆ transactions │
│ ---      ┆ ---          │
│ str      ┆ u32          │
╞══════════╪══════════════╡
│ Alpha    ┆ 9060         │
│ Bravo    ┆ 8255         │
│ Charlie  ┆ 1882         │
│ Delta    ┆ 859          │
│ Other    ┆ 422          │
│ Category ┆ 20030        │
└──────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x105be8170>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='transactions', type=<ColInfoTypeEnum.default: 1>, column_label='# Of Transactions', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10f78f790>, _spanners=Spanners([]), _heading=Heading(title='Purchase Occasions by Buyers', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[0]), grpname=None, colname='transactions', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7547a3')]), StyleInfo(locname=LocBody(columns='transactions', rows=[1]), grpname=None, colname='transactions', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8258ac')]), StyleInfo(locname=LocBody(columns='transactions', rows=[2]), grpname=None, colname='transactions', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e7e0ef')]), StyleInfo(locname=LocBody(columns='transactions', rows=[3]), grpname=None, colname='transactions', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f8f5fa')]), StyleInfo(locname=LocBody(columns='transactions', rows=[4]), grpname=None, colname='transactions', rownum=4, colnum=None, styles=[CellStyleText(color='#0

In [37]:
panel_size = 5021

# Penetration is simply the number of brand buyers divided by the number of panellists, expressed as a percentage
penetration = buyers.with_columns(
    (pl.col('buyers') / panel_size).alias('penetration')
)

# Purchases per buyer (PPB) is simply to total number of purchases occasions divided by the number of buyers.
ppb = transactions.with_columns(
    (pl.col('transactions') / penetration.get_column('buyers')).alias('ppb')
)

In [38]:
(
    GT(penetration, rowname_col='brand')
    .tab_header(title='Penetration by Category & Brand')
    .tab_stubhead(label="Brands/Category")
    .fmt_percent()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[0, 1],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(penetration='Penetration (%)')
    .cols_hide('buyers')
)

GT(_tbl_data=shape: (6, 3)
┌──────────┬────────┬─────────────┐
│ brand    ┆ buyers ┆ penetration │
│ ---      ┆ ---    ┆ ---         │
│ str      ┆ u32    ┆ f64         │
╞══════════╪════════╪═════════════╡
│ Alpha    ┆ 2624   ┆ 0.522605    │
│ Bravo    ┆ 2562   ┆ 0.510257    │
│ Charlie  ┆ 813    ┆ 0.16192     │
│ Delta    ┆ 380    ┆ 0.075682    │
│ Other    ┆ 176    ┆ 0.035053    │
│ Category ┆ 4574   ┆ 0.910974    │
└──────────┴────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x11aa96820>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='buyers', type=<ColInfoTypeEnum.hidden: 4>, column_label='buyers', column_align='center', column_width=None), ColInfo(var='penetration', type=<ColInfoTypeEnum.default: 1>, column_label='Penetration (%)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa55750>, _spanners=Spanners([]), _heading=Heading(title='Penetration by Category & Brand', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[0]), grpname=None, colname='buyers', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[1]), grpname=None, colname='buyers', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[2]), grpname=None, colname='buyers', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[3]), grpname=None, colname='buyers', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(colo

In [39]:
(
    GT(ppb, rowname_col='brand')
    .tab_header(title='Purchases per Buyer (PPB)')
    .tab_stubhead(label="Brands/Category")
    .fmt_number()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[0, 5],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(ppb='PPB')
    .cols_hide('transactions')
)

GT(_tbl_data=shape: (6, 3)
┌──────────┬──────────────┬──────────┐
│ brand    ┆ transactions ┆ ppb      │
│ ---      ┆ ---          ┆ ---      │
│ str      ┆ u32          ┆ f64      │
╞══════════╪══════════════╪══════════╡
│ Alpha    ┆ 9060         ┆ 3.452744 │
│ Bravo    ┆ 8255         ┆ 3.222092 │
│ Charlie  ┆ 1882         ┆ 2.314883 │
│ Delta    ┆ 859          ┆ 2.260526 │
│ Other    ┆ 422          ┆ 2.397727 │
│ Category ┆ 20030        ┆ 4.379099 │
└──────────┴──────────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x11aa82510>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='transactions', type=<ColInfoTypeEnum.hidden: 4>, column_label='transactions', column_align='center', column_width=None), ColInfo(var='ppb', type=<ColInfoTypeEnum.default: 1>, column_label='PPB', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa55150>, _spanners=Spanners([]), _heading=Heading(title='Purchases per Buyer (PPB)', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[0]), grpname=None, colname='transactions', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[1]), grpname=None, colname='transactions', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[2]), grpname=None, colname='transactions', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[3]), grpname=None, colname='transactions', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=Non

We see that 91% of the households in the panel purchased in the category at least once in year 1. (This is a widely purchased product category.) On average, they purchased in the category 4.4 times that year. Looking at Alpha, we see that 52% of the households in the panel purchased the brand at least once, purchasing it on average 3.5 times.

Penetration and PPB are in fact summary measures of an important but frequently overlooked summary of buyer behaviour: the **distribution of purchase frequency**. We first explore how to create this summary of category purchasing and then consider how to create such a summary of brand purchasing.

#### Distribution of Purchase

##### Distribution of Category Purchase Frequency

Looking at the *panellist-level transaction summary*, the distribution of category purchasing is determined by counting how many households made one category purchase (panellists 3102011, 3102046, etc.), two category purchases (panellists 3102012, 3102021, etc.), and so on.

In [40]:
# Category Transaction Distribution
category_trans_dist = (
    trans_summary('Category', grocery_lf, 1)
    .group_by(pl.col('# of Purchases'))
    .agg(pl.len().alias("Frequency"))
    .collect() 
    # Add zero purchase row to the top of the data series with a new column name 'Frequency'
    .pipe(
        lambda df: df.vstack(
            pl.DataFrame(
                {
                    "# of Purchases": [0],
                    "Frequency": [panel_size - df["Frequency"].sum()],
                }
            ).with_columns(
                pl.col("# of Purchases").cast(pl.UInt32),
                pl.col('Frequency').cast(pl.UInt32)
            )
        )
    )
    .sort(by='# of Purchases')
    # Add percentage of total column
    .with_columns(
        (pl.col('Frequency') / pl.col('Frequency').sum()).alias("% of Total")
    )    
)

(
    GT(category_trans_dist, rowname_col='# of Purchases')
    .tab_header(title='Category Purchase Distribution')
    .tab_stubhead(label="# of Purchases")
    .fmt_integer(columns='Frequency')
    .fmt_percent(columns='% of Total')
    .data_color(
        columns=['% of Total'],
        domain=[0, 0.2],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (24, 3)
┌────────────────┬───────────┬────────────┐
│ # of Purchases ┆ Frequency ┆ % of Total │
│ ---            ┆ ---       ┆ ---        │
│ u32            ┆ u32       ┆ f64        │
╞════════════════╪═══════════╪════════════╡
│ 0              ┆ 447       ┆ 0.089026   │
│ 1              ┆ 667       ┆ 0.132842   │
│ 2              ┆ 759       ┆ 0.151165   │
│ 3              ┆ 757       ┆ 0.150767   │
│ 4              ┆ 612       ┆ 0.121888   │
│ …              ┆ …         ┆ …          │
│ 19             ┆ 2         ┆ 0.000398   │
│ 20             ┆ 4         ┆ 0.000797   │
│ 22             ┆ 1         ┆ 0.000199   │
│ 25             ┆ 1         ┆ 0.000199   │
│ 27             ┆ 1         ┆ 0.000199   │
└────────────────┴───────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x11aabd310>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='center', column_width=None), ColInfo(var='Frequency', type=<ColInfoTypeEnum.default: 1>, column_label='Frequency', column_align='center', column_width=None), ColInfo(var='% of Total', type=<ColInfoTypeEnum.default: 1>, column_label='% of Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x105bd46e0>, _spanners=Spanners([]), _heading=Heading(title='Category Purchase Distribution', subtitle=None, preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='% of Total', rows=[0]), grpname=None, colname='% of Total', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#bba4d2')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[1]), grpname=None, colname='% of Total', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9978bb')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[2]), grpname=None, colname='% of Total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8b65b2')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[3]), grpname=None, colname='% of Total', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8c65b2')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[4]), grpname=None, colname='% of Total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#a283c1')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[5]), grpname=None, colname='% of Total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b197cb')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[6]), grpname=None, colname='% of Total', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c5b2d8')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[7]), grpname=None, colname='% of Total', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, wei

If we wish to create a visual representation of this distribution, it is tempting to simply plot the data in % of total column. However, the resulting plot would be misleading as *some purchase frequencies are missing in the data*. In particular, we see that no one made *21 category purchases; ditto for 23, 24, and 26*. One solution is to insert manually the missing number of purchases levels with 0 frequencies and then plot the data. However, the observed (relative) frequencies in the right tail are so small that they do not show up in a plot. We can therefore create a **right-censored distribution**. Here we have chosen 15 as the (right) censoring point; we see that 51 households (or 1% of the panellists) made 15 or more category purchases in year 1.

There is nothing magical about our choice of 15. Generally, the choice of censoring point is a function of how many bins you wish to display and the height of the right-most bar with which you feel comfortable.

Note that while the right-censored distribution is useful when creating a summary figure or table, it is of limited value beyond that. For example, it is not possible to compute the **mean purchase frequency** (and therefore PPB) from this summary.

In [41]:
# Create a custom bin list (with an overflow bin) to label categories
labels = [str(i) if i < 15 else '15+' for i in range(0, 16, 1)]
chart = freq_dist_plot(
    data=category_trans_dist,
    column="# of Purchases",
    bin_edges=np.arange(0, 15, 1),
    labels=labels,
    x_title="Number of Category Transactions",
    y_title="% of Households",
    chart_title="Category Transaction Distribution (%)",
    subtitle="Distribution of category purchasing in year 1",
    left_closed=False,
    compute_rel_freq=False
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


We see that 9% of the panellists made no category purchases, 13% of the panellists made one category purchase, . . . , and 1% of the panellists made at least 15 category purchases in year 1.

##### The Relationship Between the Mean and PPB

What is the average number of times the category was purchased by a household in year 1?

- Recall that mean, which we denote by $E(X)$, is given by:
  - $$E(X)=\sum\frac{xf_{x}}{n}$$
  - where $f_{x}$ is the frequency with which $x$ occurs in the dataset, $n$ is the sample size, and the summation is over all possible values of $x$.
- Denoting the relative frequency with which $x$ occurs (i.e., $f_{x}/n$) by $P(X = x)$, the mean is given by:
  - $$E(X)=\sum x P(X = x)$$
  - More formally, we can say that $P(X = x)$ is the empirical probability that a randomly chosen household made $x$ purchases.

In [42]:
# Mean of the relative purchase frequency
mean = category_trans_dist.select(
    (pl.col('# of Purchases') * pl.col('% of Total')).sum()
).item(0,0)
print(f'{mean = :0.3f}')

mean = 3.989


We see that average number of category purchases is 4.0. Why is this different from the 4.4 purchases per buyer (PPB) computed above? The mean we have just computed *includes those households that made zero purchases*, whereas *PPB is the average among those households that made at least one* (in this case category) purchase.

We can derive the relationship between these two quantities (mean of the relative frequency and PPB) in the following manner:
$$
  \begin{aligned}
    PBB &= \sum_{x=1}^{\max x} \frac{xf_{x}}{n-f_{0}} \\
                   &= \sum_{x=0}^{\max x} \frac{xf_{x}}{n-f_{0}} \\
                   &= \sum_{x=0}^{\max x} \left(\frac{xf_{x}}{n-f_{0}}\right)\left(\frac{n}{n}\right) \\
                   &= \sum_{x=0}^{\max x} \left(\frac{xf_{x}}{n}\right)\left(\frac{n}{n-f_{0}}\right) \\
                   &= \left(\frac{n}{n-f_{0}}\right)\sum_{x=0}^{\max x} \left(\frac{xf_{x}}{n}\right) \\
                   &= \frac{E(X)}{1-P(X=0)}
  \end{aligned}
$$

In other words, **PPB is the mean divided by penetration**.

In [43]:
# All category PPB computations here are equivalent
ppb_category = mean / penetration.filter(pl.col('brand') == 'Category').item(0,2)
print(ppb_category)
print(ppb.filter(pl.col('brand') == 'Category').item(0,2))
ppb_category = mean / (1 - category_trans_dist['% of Total'][0])
print(ppb_category)

4.379099256668124
4.379099256668124
4.379099256668124


##### Distribution of Brand Purchase Frequency

Let us now create the distribution of purchase frequency for Alpha, which is. With the one exception noted below, the logic follows that associated with creating the distribution of category purchasing.

In contrast to the `groupby` aggregation output associated with our summary of category purchasing, this aggregation does contain a zero category. However, we must be careful in our interpretation of the associated frequency. We see that 1950 category buyers did not buy Alpha in year 1. However, in order to have a complete summary of brand purchasing, we should also account for those 447 households that made no category purchases that year. The number of panellists making zero purchases of Alpha is the total number of panellists (5021) minus the number of panellists that made 1, 2, 3, . . . purchases.

**Note**: What is Alpha’s penetration of category buyers? (1 − 1950/4574) = 57%.
- Category Penetration = (1 - % of category buyers who did not purchase the brand)

In [44]:
# Alpha Brand Transaction Distribution
alpha_trans_dist = (
    trans_summary('Alpha', grocery_lf, 1)
    .group_by(pl.col('# of Purchases'))
    .agg(pl.len().alias("Frequency"))
    .collect() 
    .pipe(
        lambda df: df.vstack(
            pl.DataFrame(
                {
                    "# of Purchases": [0],
                    # Calculate zero purchases by subtracting the total sum of all purchases from the full panel size
                    "Frequency": [panel_size - df["Frequency"].sum()], # This ensures all panelists who made zero purchases are accounted for
                }
            ).with_columns(
                pl.col("# of Purchases").cast(pl.UInt32),
                pl.col('Frequency').cast(pl.UInt32)
            )
        )
    )
    .sort(by='# of Purchases')
    .with_columns(
        # Add a '% of Total' column
        (pl.col('Frequency') / pl.col('Frequency').sum()).alias("% of Total")
    )    
)

(
    GT(alpha_trans_dist, rowname_col='# of Purchases')
    .tab_header(title='Alpha Purchase Distribution')
    .tab_stubhead(label="# of Purchases")
    .fmt_integer(columns='Frequency')
    .fmt_percent(columns='% of Total')
    .data_color(
        columns=['% of Total'],
        domain=[0, 0.5],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 3)
┌────────────────┬───────────┬────────────┐
│ # of Purchases ┆ Frequency ┆ % of Total │
│ ---            ┆ ---       ┆ ---        │
│ u32            ┆ u32       ┆ f64        │
╞════════════════╪═══════════╪════════════╡
│ 0              ┆ 2397      ┆ 0.477395   │
│ 1              ┆ 733       ┆ 0.145987   │
│ 2              ┆ 517       ┆ 0.102968   │
│ 3              ┆ 400       ┆ 0.079665   │
│ 4              ┆ 277       ┆ 0.055168   │
│ …              ┆ …         ┆ …          │
│ 14             ┆ 12        ┆ 0.00239    │
│ 15             ┆ 6         ┆ 0.001195   │
│ 16             ┆ 3         ┆ 0.000597   │
│ 18             ┆ 1         ┆ 0.000199   │
│ 20             ┆ 1         ┆ 0.000199   │
└────────────────┴───────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x11aabd720>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='center', column_width=None), ColInfo(var='Frequency', type=<ColInfoTypeEnum.default: 1>, column_label='Frequency', column_align='center', column_width=None), ColInfo(var='% of Total', type=<ColInfoTypeEnum.default: 1>, column_label='% of Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10f7b5310>, _spanners=Spanners([]), _heading=Heading(title='Alpha Purchase Distribution', subtitle=None, preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='% of Total', rows=[0]), grpname=None, colname='% of Total', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#6d3c9e')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[1]), grpname=None, colname='% of Total', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d2c3e1')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[2]), grpname=None, colname='% of Total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#dfd5ea')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[3]), grpname=None, colname='% of Total', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e7deef')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[4]), grpname=None, colname='% of Total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#eee8f4')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[5]), grpname=None, colname='% of Total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f1edf6')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[6]), grpname=None, colname='% of Total', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f6f3f9')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[7]), grpname=None, colname='% of Total', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight

In [45]:
# Create a custom bin list (with an overflow bin) to label categories
labels = [str(i) if i < 10 else '10+' for i in range(11)]

chart = freq_dist_plot(
    data=alpha_trans_dist,
    column="# of Purchases",
    bin_edges=np.arange(0, 10, 1),
    labels=labels,
    x_title="Number of Transactions",
    y_title="% of Households",
    chart_title="Alpha Transaction Distribution (%)",
    subtitle="Distribution of purchase frequency for Alpha in year 1",
    left_closed=False,
    compute_rel_freq=False
)
chart.show()


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Distribution of Spend

##### Distribution of Category Spend

We now turn our attention to creating summaries of total spend. Our initial goal is to create a histogram of category spend (in dollars) across those panellists that made at least one purchase in the category in year 1. In the following plot, the raw total spend data have been binned in bins with a width of $5. 

We see that 16% of category buyers spent up to $5 in the category during this one-year period; 23% spent between $5 and $10; . . . and 2% spent more than $50.

Before deciding on what bin width to use when creating the histogram, let us first compute some basic descriptive statistics.

In [46]:
category_spend = spend_summary('Category', grocery_lf, 1).drop('panel_id')

category_spend.describe()

statistic,spend
str,f64
"""count""",4574.0
"""null_count""",0.0
"""mean""",15.763424
"""std""",12.876024
"""min""",1.27
"""25%""",6.76
"""50%""",12.57
"""75%""",20.74
"""max""",166.699936


We wish to visualize the variability in category spend. Given this objective, some would automatically think of creating a (kernel) density plot.

In [47]:
alt.Chart(category_spend.collect()).transform_density(
    'spend',
    as_=['spend', 'Density']
).mark_area().encode(
    x=alt.X("spend:Q").scale(domain=(0,150)),
    y='Density:Q',
).properties(
    width=650,
    height=250
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


While this provides a good visualisation of the shape of the distribution, it can be difficult for most “consumers” of the plot to extract some additional information that may be of interest. For example, it is not easy to answer the question “What percentage of category buyers spent $30 or less in year 1?”.

There appears to be quite a bit of variability in category spend. To get further insight into the distribution of total category spend across the panellists, we need to determine the total spend quantities associate with the various percentile levels.

In [48]:
percentiles = np.arange(5, 96, 5)
# np.percentile returns the k-th percentile of values in a range
percentile_values = np.percentile(category_spend.select('spend').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Category Spend Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 40],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 2.69       │
│ 10               ┆ 3.39       │
│ 15               ┆ 4.67       │
│ 20               ┆ 5.88       │
│ 25               ┆ 6.76       │
│ …                ┆ …          │
│ 75               ┆ 20.7375    │
│ 80               ┆ 23.43      │
│ 85               ┆ 26.880501  │
│ 90               ┆ 31.809999  │
│ 95               ┆ 39.718504  │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x11aa797c0>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa65550>, _spanners=Spanners([]), _heading=Heading(title='Category Spend Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ece6f2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cce6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c6b2d9')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b299cc')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9f80bf')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8c66b2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#794ca6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percenti

We see that 5% of the category buyers spent $2.69 or less in the category during the year, 10% spent $3.39 or less, and so on. The heaviest 5% of buyers each spent more than $39.72 in the category during the year.

Looking out this output, we conclude that a bin size of $5 is probably about right.

How many bins do we go with? This is an empirical question. Since 5% of the panel spent more than $39.72, we certainly want to go above $40 in order to get a sense of how they are spread between $39.72 and the maximum of $166.70. We will go with $50. If too many panellists have spent more than $50, we can always add more bins. If too few panellists fall into this bin, we can always combine the bins we have created.

**Note**: The distributions of many customer behaviours have a long right tail. Accommodating the range of values can make it difficult to get a clear sense of what is happening on the left side of the distribution. It can therefore be helpful to bin the data (as with a histogram) but to right censor the data, assigning all of the observations with a value of $𝑥$ or higher to an $𝑥 +$ bin

We see that 750 panellists spent $5 or less in the category during year 1, 1073 spent between $5 and $10, . . . , and 109 spent more than $50.

We convert the raw counts into percentages and plot these percentages as a bar chart.

In [49]:
# Create a custom bin list (with an overflow bin) to label categories
labels = [f"{lb} - {lb + 5 if lb != 50 else '∞'}" for lb in range(-5, 55, 5)]
chart = freq_dist_plot(
    data=category_spend,
    column="spend",
    bin_edges=np.arange(0, 51, 5),
    labels=labels,
    x_title="Category Spend ($)",
    y_title="% of Households",
    chart_title="Category Spend Distribution (%)",
    subtitle="Distribution of category spend across category buyers"
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


The general shape of this distribution (i.e., an interior mode, median less than the mean, right-skewed with a long right-tail) is what we typically observe when we look at spend data.

This, of course, depends on the bin width we choose when summarising the data. If we had chosen a bin width of $10, the left-most bar would be the highest bar and we would no longer observe an interior mode.

##### Distribution of Brand Spend

Let us now create the distribution of spend on Alpha. We will follow the same basic process as for the distribution of category spend with a few minor changes.

Note that there are a number of 0 or NaN values in the Alpha spend column. While these panellists purchased in the category during the year, they did not purchase any of Alpha’s SKUs. The first thing we need to do is remove these observations.

In [50]:
alpha_spend = spend_summary('Alpha', grocery_lf, 1).drop('panel_id')

alpha_spend.describe()

statistic,spend
str,f64
"""count""",2624.0
"""null_count""",0.0
"""mean""",12.793803
"""std""",11.715616
"""min""",1.57
"""25%""",3.69
"""50%""",9.67
"""75%""",17.130001
"""max""",105.899971


In [51]:
percentiles = np.arange(5, 96, 5)

percentile_values = np.percentile(alpha_spend.select('spend').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Alpha Spend Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 40],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 2.453      │
│ 10               ┆ 2.79       │
│ 15               ┆ 3.38       │
│ 20               ┆ 3.49       │
│ 25               ┆ 3.69       │
│ …                ┆ …          │
│ 75               ┆ 17.130001  │
│ 80               ┆ 20.040001  │
│ 85               ┆ 23.1355    │
│ 90               ┆ 27.62      │
│ 95               ┆ 35.485003  │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x11aa7a620>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa65710>, _spanners=Spanners([]), _heading=Heading(title='Alpha Spend Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ece6f2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cce6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c6b2d9')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b299cc')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9f80bf')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8c66b2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#794ca6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percentile 

Having looked at some basic descriptive stats (as above), we will summarise the data using bins of width $2, right-censoring at $40.

In [52]:
labels = [f"{lb} - {lb + 2 if lb != 40 else '∞'}" for lb in range(-2, 41, 2)]
chart = freq_dist_plot(
    data=alpha_spend,
    column="spend",
    bin_edges=np.arange(0, 41, 2),
    labels=labels,
    x_title="Brand Spend ($)",
    y_title="% of Households",
    chart_title="Alpha Spend Distribution (%)",
    subtitle="Distribution of spend on Alpha across brand buyers",
    label_angle=-45
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


The general observation made earlier about category spend distribution about the shape of the distribution of spend holds. It is not so smooth, but this is a function of the smaller bins sizes. There is, however, one obvious aberration—the large spike for \$2–\$4. What is going on here? The average price of most Alpha SKUs is between \$2 and \$4. (Review average SKU prices of Alpha) We recall from Alpha's distribution of purchase frequency plot that 48% of the panel made zero purchases of Alpha and 15% made one transaction. This implies that 0.15/(1 − 0.48) = 29% of Alpha buyers made just one purchase of the brand. Assuming they only purchased only one unit of one Alpha SKU on that purchase occasion, we would expect a large number of Alpha buyers to spend between \$2 and \$4; 24% is not unrealistic.

How would you determine the validity of the assumption that the buyers who purchased Alpha just once mostly bought only one unit of Alpha SKU on that purchase occasion?

##### Analyzing Alpha's One-Time Buyers in Year 1

In [53]:
# One-Time Alpha Buyers
one_time_alpha_buyers = trans_summary('Alpha',grocery_lf,1).filter(pl.col('# of Purchases') == 1).collect()
one_time_alpha_buyers

# Alpha buyers who spent between $2 to $4
spend_range_alpha_buyers = spend_summary('Alpha', grocery_lf, 1).filter(
    (pl.col('spend') >= 2) &
    (pl.col('spend') < 4)
).collect()

res = one_time_alpha_buyers.with_columns(
    contains=pl.col('panel_id').is_in(spend_range_alpha_buyers.select('panel_id'))
).select('contains').sum().item()

print(f'There were {one_time_alpha_buyers.count().item(0,'panel_id')} one-time buyers of Alpha brand in Year 1.')
print(f'{res} of those one-time Alpha buyers spent between [$2, $4).')
print(f'Given that there are {spend_range_alpha_buyers.count().item(0,'spend')} Alpha buyers in total who spent between $2-$4,')
print(f'the {res} one-time buyers represent {res / spend_range_alpha_buyers.count().item(0,'spend'):.0%} of the Alpha buyers in that spend range.')

There were 733 one-time buyers of Alpha brand in Year 1.
623 of those one-time Alpha buyers spent between [$2, $4).
Given that there are 633 Alpha buyers in total who spent between $2-$4,
the 623 one-time buyers represent 98% of the Alpha buyers in that spend range.


**Validate**:

In [54]:
spend_range_alpha_buyers.filter(
    pl.col('panel_id').is_in(one_time_alpha_buyers.select('panel_id'))
).select(pl.col('spend').sum()).item()

1937.22998046875

In [55]:
grocery_lf.filter(
    (pl.col('week') <= 52) &
    (pl.col('panel_id').is_in(spend_range_alpha_buyers.select('panel_id'))) &  # Assuming panel_id is the relevant column
    (pl.col('panel_id').is_in(one_time_alpha_buyers.select('panel_id'))) &  # Same assumption
    (pl.col('brand') == 'Alpha')
).with_columns(
    ((pl.col('units') * pl.col('price'))).alias('spend')
).select(pl.col('spend').sum()).collect().item()

1937.229736328125

#### Distribution of Volume Purchase

##### Distribution of Category Volume Purchase Frequency

In [56]:
category_vol = vol_summary('Category', grocery_lf, 1).drop('panel_id')

category_vol.describe()

statistic,volume
str,f64
"""count""",4574.0
"""null_count""",0.0
"""mean""",4.544163
"""std""",3.884697
"""min""",0.25
"""25%""",2.0
"""50%""",3.5
"""75%""",6.0
"""max""",67.0


In [57]:
percentiles = np.arange(5, 96, 5)

with pl.StringCache(): 
    percentile_values = np.percentile(category_vol.select('volume').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Category Volume Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 15],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 0.8325     │
│ 10               ┆ 1.0        │
│ 15               ┆ 1.0        │
│ 20               ┆ 1.5        │
│ 25               ┆ 2.0        │
│ …                ┆ …          │
│ 75               ┆ 6.0        │
│ 80               ┆ 7.0        │
│ 85               ┆ 8.0        │
│ 90               ┆ 9.0        │
│ 95               ┆ 12.0       │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x11aa7ac60>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10f796b60>, _spanners=Spanners([]), _heading=Heading(title='Category Volume Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ccbbdd')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9977bb')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percent

In [58]:
labels = [f"{lb} - {lb + 1 if lb != 12 else '∞'}" for lb in range(-1, 13, 1)]
chart = freq_dist_plot(
    data=category_vol,
    column="volume",
    bin_edges=np.arange(0, 13, 1),
    labels=labels,
    x_title="Category Volume Purchase ($)",
    y_title="% of Households",
    chart_title="Category Volume Purchase Distribution (%)",
    subtitle="Distribution of category volume purchases across category buyers"
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


##### Distribution of Brand Volume Purchase Frequency

Note that there are a number of 0 or NaN values in the Alpha volume purchase column. These panaellists (like the brand spend section) purchased in the category during the year, but they did not purchase any of Alpha's products. Let us remove observations of `panel_ids` that have not at least made 1 purchase from Alpha brand during the year.

In [59]:
alpha_vol = vol_summary('Alpha', grocery_lf, 1).drop('panel_id')

alpha_vol.describe()

statistic,volume
str,f64
"""count""",2624.0
"""null_count""",0.0
"""mean""",3.493236
"""std""",3.350901
"""min""",0.25
"""25%""",1.0
"""50%""",2.5
"""75%""",5.0
"""max""",27.55


In [60]:
percentiles = np.arange(5, 96, 5)

with pl.StringCache(): 
    percentile_values = np.percentile(alpha_vol.select('volume').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Alpha Volume Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 15],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 0.5        │
│ 10               ┆ 0.5        │
│ 15               ┆ 1.0        │
│ 20               ┆ 1.0        │
│ 25               ┆ 1.0        │
│ …                ┆ …          │
│ 75               ┆ 5.0        │
│ 80               ┆ 5.5        │
│ 85               ┆ 6.35       │
│ 90               ┆ 8.0        │
│ 95               ┆ 10.0       │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x11b5e1360>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aaad910>, _spanners=Spanners([]), _heading=Heading(title='Alpha Volume Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ccbbdd')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9977bb')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percentile

In [61]:
labels = [f"{lb} - {lb + 1 if lb != 9 else '∞'}" for lb in range(-1, 10, 1)]

chart = freq_dist_plot(
    data=alpha_vol,
    column="volume",
    bin_edges=np.arange(0, 10, 1),
    labels=labels,
    x_title="Brand Volume Purchase ($)",
    y_title="% of Households",
    chart_title="Alpha Brand Volume Purchase Distribution (%)",
    subtitle="Distribution of Alpha volume purchases across brand buyers"
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Garbage Collect

In [62]:
exceptions = ['grocery_lf', 'sku_lf', 'kiwi_lf', 'In', 'exceptions', 'active_variables',
              'penetration', 'ppb', 'panel_size']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (pl.LazyFrame, pl.DataFrame, pl.Series, alt.Chart, alt.LayerChart, list, int, float, str, np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var

gc.collect()

1364

#### Performing a Basic Decile Analysis

We have observed that there is a lot of variability in purchase frequency and spend, be it at the product or category level. A useful way of expressing the variability is via a decile analysis. As the name suggests, this sees us **dividing the customer base into 10 equally sized groups and then summarising each group’s buying behaviour**. We will consider two versions of this analysis.

##### Decile analysis of category buying behaviour - Decile based on category buyers

We will first focus on creating the table below. The table captures the decile analysis of category buying behaviour (where each decile equals 10% category buyers). Recall that 4574 households made at least one category purchase in year 1. Having **sorted these households by total category spend**, we create **10 equally sized groups**. Decile 1 is the 10% of households that spent the most in the category during year 1, decile 2 is the next largest 10% of spenders, and so on.

In [63]:
# Create decile cuts in the group_spend data using the sorted and ranked data

# Part 1: Break-tie ranking of total category spend data
# Part 2: Decile cuts in ranking
buyer_decile = spend_summary(
    'Category', grocery_lf, 1
).with_columns(
    pl.col('spend').rank(method='ordinal', descending=True).alias('rank'),
).with_columns(
    (np.floor(10 * (pl.col('rank') - 1) / pl.col('rank').max()) + 1).cast(pl.UInt16).alias('decile')
).sort('decile', 'rank').collect()

buyer_decile

panel_id,spend,rank,decile
u32,f32,u32,u16
3111594,166.699936,1,1
3119170,108.829994,2,1
3125418,102.87001,3,1
3110132,100.630013,4,1
3109647,97.37999,5,1
…,…,…,…
3128086,1.49,4570,10
3112878,1.39,4571,10
3120986,1.33,4572,10


In [64]:
# Helper functions
def calculate_percentage(series):
    return series / series.sum()

def group_aggregate(data, group_col, agg_col, agg_expr):
    return data.group_by(group_col).agg(agg_expr(pl.col(agg_col)))

# Decile table creation
decile_tab = buyer_decile.select("decile").unique().sort("decile").rename({"decile": "Decile"})

# Populate decile statistics
# Decile counts and % HH
decile_counts = group_aggregate(buyer_decile, 'decile', 'panel_id', pl.Expr.count)
decile_tab.insert_column(1, pl.lit(calculate_percentage(decile_counts['panel_id'])).alias('% HH')) 

# % Spend
decile_spend = group_aggregate(buyer_decile, 'decile', 'spend', pl.Expr.sum)
decile_tab.insert_column(2, pl.lit(calculate_percentage(decile_spend['spend'])).alias('% Spend')) 

# % Transactions
group_trans = trans_summary('Category', grocery_lf, 1).collect().join(
    other=buyer_decile.group_by('panel_id').agg(pl.col('decile').sum()),
    on='panel_id'
).sort('decile')
decile_trans = group_aggregate(group_trans, 'decile', '# of Purchases', pl.Expr.sum)
decile_tab.insert_column(3, pl.lit(calculate_percentage(decile_trans['# of Purchases'])).alias('% Trans')) 

# Spend/HH, Cat Trans/HH, Avg Spend/Trans (AOV)
decile_tab.insert_column(4, pl.lit(decile_spend['spend'] / decile_counts['panel_id']).alias('Spend/HH'))
decile_tab.insert_column(5, pl.lit(decile_trans['# of Purchases'] / decile_counts['panel_id']).alias('Cat Trans/HH'))
decile_tab.insert_column(6, pl.lit(decile_spend['spend'] / decile_trans['# of Purchases']).alias('Avg Spend/Trans'))

# Unique Brands Purchased
unique_brands = group_aggregate(group_trans, 'decile', 'Brands Purchased', pl.Expr.mean)
decile_tab.insert_column(7, pl.lit(unique_brands['Brands Purchased']).alias('# Unique Brands'))

(
    GT(decile_tab, rowname_col='Decile')
    .tab_header(title='Decile Analysis of Category Buying Behavior',
                subtitle= 'Each decile equals 10% of category buyers')
    .tab_stubhead(label="Decile")
    .fmt_percent(columns=['% HH', '% Spend', '% Trans'], decimals=1)
    .fmt_number(columns=['Cat Trans/HH', '# Unique Brands'])
    .fmt_currency(columns=['Spend/HH','Avg Spend/Trans'])
)

GT(_tbl_data=shape: (10, 8)
┌────────┬──────────┬──────────┬──────────┬───────────┬──────────────┬──────────────────┬──────────┐
│ Decile ┆ % HH     ┆ % Spend  ┆ % Trans  ┆ Spend/HH  ┆ Cat Trans/HH ┆ Avg Spend/Trans  ┆ # Unique │
│ ---    ┆ ---      ┆ ---      ┆ ---      ┆ ---       ┆ ---          ┆ ---              ┆ Brands   │
│ u16    ┆ f64      ┆ f32      ┆ f64      ┆ f64       ┆ f64          ┆ f64              ┆ ---      │
│        ┆          ┆          ┆          ┆           ┆              ┆                  ┆ f64      │
╞════════╪══════════╪══════════╪══════════╪═══════════╪══════════════╪══════════════════╪══════════╡
│ 1      ┆ 0.100131 ┆ 0.283613 ┆ 0.242786 ┆ 44.648476 ┆ 10.617904    ┆ 4.205018         ┆ 1.847162 │
│ 2      ┆ 0.099913 ┆ 0.172067 ┆ 0.163005 ┆ 27.147378 ┆ 7.14442      ┆ 3.799801         ┆ 1.636761 │
│ 3      ┆ 0.100131 ┆ 0.133154 ┆ 0.134548 ┆ 20.962076 ┆ 5.884279     ┆ 3.562386         ┆ 1.582969 │
│ 4      ┆ 0.099913 ┆ 0.107161 ┆ 0.111083 ┆ 16.906979 ┆ 4.868709     ┆ 3.472579         ┆ 1.540481 │
│ 5      ┆ 0.099913 ┆ 0.08755  ┆ 0.094708 ┆ 13.812933 ┆ 4.150985     ┆ 3.327628         ┆ 1.498906 │
│ 6      ┆ 0.100131 ┆ 0.070741 ┆ 0.077883 ┆ 11.136638 ┆ 3.406114     ┆ 3.269603         ┆ 1.412664 │
│ 7      ┆ 0.099913 ┆ 0.056896 ┆ 0.067049 ┆ 8.976631  ┆ 2.938731     ┆ 3.054594         ┆ 1.374179 │
│ 8      ┆ 0.100131 ┆ 0.042816 ┆ 0.048777 ┆ 6.740415  ┆ 2.133188     ┆ 3.159785         ┆ 1.224891 │
│ 9      ┆ 0.099913 ┆ 0.028881 ┆ 0.036895 ┆ 4.556586  ┆ 1.617068     ┆ 2.817808         ┆ 1.212254 │
│ 10     ┆ 0.099913 ┆ 0.017122 ┆ 0.023265 ┆ 2.701423  ┆ 1.019694     ┆ 2.649249         ┆ 1.0      │
└────────┴──────────┴──────────┴──────────┴───────────┴──────────────┴──────────────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x11b55a0d0>, _boxhead=Boxhead([ColInfo(var='Decile', type=<ColInfoTypeEnum.stub: 2>, column_label='Decile', column_align='center', column_width=None), ColInfo(var='% HH', type=<ColInfoTypeEnum.default: 1>, column_label='% HH', column_align='right', column_width=None), ColInfo(var='% Spend', type=<ColInfoTypeEnum.default: 1>, column_label='% Spend', column_align='right', column_width=None), ColInfo(var='% Trans', type=<ColInfoTypeEnum.default: 1>, column_label='% Trans', column_align='right', column_width=None), ColInfo(var='Spend/HH', type=<ColInfoTypeEnum.default: 1>, column_label='Spend/HH', column_align='right', column_width=None), ColInfo(var='Cat Trans/HH', type=<ColInfoTypeEnum.default: 1>, column_label='Cat Trans/HH', column_align='right', column_width=None), ColInfo(var='Avg Spend/Trans', type=<ColInfoTypeEnum.default: 1>, column_label='Avg Spend/Trans', column_align='right', column_width=None), ColInfo(var='# Unique Brands', type=<ColInfoTypeEnum.default: 1>, column_label='# Unique Brands', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10f67abd0>, _spanners=Spanners([]), _heading=Heading(title='Decile Analysis of Category Buying Behavior', subtitle='Each decile equals 10% of category buyers', preheader=None), _stubhead='Decile', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x11aaadb50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x11aa42b70>, <great_tables._gt_data.FormatInfo object at 0x11aa43890>, <great_tables._gt_data.FormatInfo object at 0x11aaa56d0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value=

Reading across the row associated with Decile 1, we see that they accounted for 28% of category spend and 24% of total transactions. On average, they spent $44.65 in the category across an average of 10.6 purchase occasions. This corresponds to an average category spend per category transaction of $4.21. On average, these households purchased 1.8 different
brands during the year. Contrast this to the 10% of category buyers that spent the least in the category. They account for 2% of category spend and transactions, making on average one category purchase. And so on.

This is a very basic decile table. Some additional information that could be reported includes the *average number of units purchased per transaction* and the *average number of unique SKUs purchased in the year*.

##### Decile analysis of category buying behaviour - Decile based on category spend/revenue

The decile analysis we have just completed uses deciles that represent 10% of the category buyers. An alternative approach is to create deciles that represent **10% of category spend**. In other words, the decile analysis took the entire customer base that purchased the category in year 1 and split the base into ten equal group. In this analysis, we will take the total customer spend or total revenue and split that revenue into ten equal groups. The only change to what we have done above is how we create the decile variable.


In [65]:
# Divide the total revenue into 10 equal parts and generate the decile bins
spend_total = spend_summary('Category', grocery_lf, 1).select(pl.col('spend')).sum().collect().to_series()[0]
breaks = np.arange(0, spend_total, spend_total/10)
labels = [f'{i}' for i in range(0, 11, 1)]

spend_decile = spend_summary(
    'Category', grocery_lf, 1
).sort(
    'spend', descending=True
).with_columns(
    pl.col('spend').cum_sum().alias('cum sum') # Create a cumulative sum column for category spend (category revenue)
).with_columns(
    pl.col('cum sum').cut( # Cut function determines the decile range for each cumulative sum value
        breaks=breaks,
        labels=labels # Function also adds the decile label from 1 to 10
    ).cast(pl.UInt16).alias('decile')
).sort('decile').collect()

spend_decile

panel_id,spend,cum sum,decile
u32,f32,f32,u16
3111594,166.699936,166.699936,1
3119170,108.829994,275.529938,1
3125418,102.87001,378.399963,1
3110132,100.630013,479.029968,1
3109647,97.37999,576.409973,1
…,…,…,…
3128086,1.49,72096.984375,10
3112878,1.39,72098.375,10
3120986,1.33,72099.703125,10


In [66]:
# Helper functions
def calculate_percentage(series):
    return series / series.sum()

def group_aggregate(data, group_col, agg_col, agg_expr):
    return data.group_by(group_col).agg(agg_expr(pl.col(agg_col)))

# Decile table creation
decile_tab_spend = spend_decile.select("decile").unique().sort("decile").rename({"decile": "Decile"})

# Populate decile statistics
# Decile counts and % HH
decile_counts = group_aggregate(spend_decile, 'decile', 'panel_id', pl.Expr.count)
decile_tab_spend.insert_column(1, pl.lit(calculate_percentage(decile_counts['panel_id'])).alias('% HH')) 

# % Spend
decile_spend = group_aggregate(spend_decile, 'decile', 'spend', pl.Expr.sum)
decile_tab_spend.insert_column(2, pl.lit(calculate_percentage(decile_spend['spend'])).alias('% Spend')) 

# % Transactions
group_trans = trans_summary('Category', grocery_lf, 1).collect().join(
    other=spend_decile.group_by('panel_id').agg(pl.col('decile').sum()),
    on='panel_id'
).sort('decile')
decile_trans = group_aggregate(group_trans, 'decile', '# of Purchases', pl.Expr.sum)
decile_tab_spend.insert_column(3, pl.lit(calculate_percentage(decile_trans['# of Purchases'])).alias('% Trans')) 

# Spend/HH, Cat Trans/HH, Avg Spend/Trans (AOV)
decile_tab_spend.insert_column(4, pl.lit(decile_spend['spend'] / decile_counts['panel_id']).alias('Spend/HH'))
decile_tab_spend.insert_column(5, pl.lit(decile_trans['# of Purchases'] / decile_counts['panel_id']).alias('Cat Trans/HH'))
decile_tab_spend.insert_column(6, pl.lit(decile_spend['spend'] / decile_trans['# of Purchases']).alias('Avg Spend/Trans'))

# Unique Brands Purchased
unique_brands = group_aggregate(group_trans, 'decile', 'Brands Purchased', pl.Expr.mean)
decile_tab_spend.insert_column(7, pl.lit(unique_brands['Brands Purchased']).alias('# Unique Brands'))

(
    GT(decile_tab_spend, rowname_col='Decile')
    .tab_header(title='Decile Analysis of Category Buying Behavior',
                subtitle= 'Each decile equals 10% of category spend/revenue')
    .tab_stubhead(label="Decile")
    .fmt_percent(columns=['% HH', '% Spend', '% Trans'], decimals=1)
    .fmt_number(columns=['Cat Trans/HH', '# Unique Brands'])
    .fmt_currency(columns=['Spend/HH','Avg Spend/Trans'])
)

GT(_tbl_data=shape: (10, 8)
┌────────┬──────────┬──────────┬──────────┬───────────┬──────────────┬──────────────────┬──────────┐
│ Decile ┆ % HH     ┆ % Spend  ┆ % Trans  ┆ Spend/HH  ┆ Cat Trans/HH ┆ Avg Spend/Trans  ┆ # Unique │
│ ---    ┆ ---      ┆ ---      ┆ ---      ┆ ---       ┆ ---          ┆ ---              ┆ Brands   │
│ u16    ┆ f64      ┆ f32      ┆ f64      ┆ f64       ┆ f64          ┆ f64              ┆ ---      │
│        ┆          ┆          ┆          ┆           ┆              ┆                  ┆ f64      │
╞════════╪══════════╪══════════╪══════════╪═══════════╪══════════════╪══════════════════╪══════════╡
│ 1      ┆ 0.024705 ┆ 0.099946 ┆ 0.075836 ┆ 63.772496 ┆ 13.442478    ┆ 4.744103         ┆ 1.920354 │
│ 2      ┆ 0.037167 ┆ 0.099811 ┆ 0.089116 ┆ 42.332709 ┆ 10.5         ┆ 4.031687         ┆ 1.823529 │
│ 3      ┆ 0.046349 ┆ 0.099887 ┆ 0.092411 ┆ 33.971984 ┆ 8.731132     ┆ 3.890903         ┆ 1.783019 │
│ 4      ┆ 0.055969 ┆ 0.100177 ┆ 0.094109 ┆ 28.214609 ┆ 7.363281     ┆ 3.831798         ┆ 1.628906 │
│ 5      ┆ 0.066463 ┆ 0.099893 ┆ 0.098602 ┆ 23.692237 ┆ 6.496711     ┆ 3.646805         ┆ 1.648026 │
│ 6      ┆ 0.078924 ┆ 0.100206 ┆ 0.102097 ┆ 20.013875 ┆ 5.66482      ┆ 3.533012         ┆ 1.573407 │
│ 7      ┆ 0.094666 ┆ 0.100073 ┆ 0.103744 ┆ 16.663905 ┆ 4.799076     ┆ 3.472315         ┆ 1.535797 │
│ 8      ┆ 0.116747 ┆ 0.099911 ┆ 0.108737 ┆ 13.490187 ┆ 4.078652     ┆ 3.307511         ┆ 1.477528 │
│ 9      ┆ 0.156756 ┆ 0.100032 ┆ 0.112282 ┆ 10.059274 ┆ 3.136681     ┆ 3.206981         ┆ 1.390516 │
│ 10     ┆ 0.322256 ┆ 0.100065 ┆ 0.123065 ┆ 4.894757  ┆ 1.67232      ┆ 2.926925         ┆ 1.162144 │
└────────┴──────────┴──────────┴──────────┴───────────┴──────────────┴──────────────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x11b55acb0>, _boxhead=Boxhead([ColInfo(var='Decile', type=<ColInfoTypeEnum.stub: 2>, column_label='Decile', column_align='center', column_width=None), ColInfo(var='% HH', type=<ColInfoTypeEnum.default: 1>, column_label='% HH', column_align='right', column_width=None), ColInfo(var='% Spend', type=<ColInfoTypeEnum.default: 1>, column_label='% Spend', column_align='right', column_width=None), ColInfo(var='% Trans', type=<ColInfoTypeEnum.default: 1>, column_label='% Trans', column_align='right', column_width=None), ColInfo(var='Spend/HH', type=<ColInfoTypeEnum.default: 1>, column_label='Spend/HH', column_align='right', column_width=None), ColInfo(var='Cat Trans/HH', type=<ColInfoTypeEnum.default: 1>, column_label='Cat Trans/HH', column_align='right', column_width=None), ColInfo(var='Avg Spend/Trans', type=<ColInfoTypeEnum.default: 1>, column_label='Avg Spend/Trans', column_align='right', column_width=None), ColInfo(var='# Unique Brands', type=<ColInfoTypeEnum.default: 1>, column_label='# Unique Brands', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa4f6a0>, _spanners=Spanners([]), _heading=Heading(title='Decile Analysis of Category Buying Behavior', subtitle='Each decile equals 10% of category spend/revenue', preheader=None), _stubhead='Decile', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x11aab3b70>, _formats=[<great_tables._gt_data.FormatInfo object at 0x11a9f70d0>, <great_tables._gt_data.FormatInfo object at 0x10f19a2d0>, <great_tables._gt_data.FormatInfo object at 0x11b544130>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value',

Reading across the row associated with Decile 1, we see that the top 2% of households accounted for 10% of category spend. On average, they spent \$63.77 in the category, across an average of 13.4 purchase occasions. This corresponds to an average category spend per category transaction of \$4.74. On average, these households purchased 1.9 different brands during the year.

#### Lorenz Curves

##### Lorenz Curves - Transactions/Purchasing - Discrete

A Lorenz curve is a common graphical tool for visualising “concentration” or “inequality” in the distribution of a quantity of interest (e.g., income, buying behaviour). It show the proportion of the overall quantity (e.g., number of transactions, spend) associated with the bottom x% of the unit of observation associated with the distribution (e.g., households). When analysing buyer behaviour, this sees us lining up all customers in ascending order of their level of purchasing and computing the share of total purchasing
accounted by each person. The Lorenz curve is created by plotting the cumulative percentage of customers (x-axis) against the cumulative percentage of total purchasing (y-axis).

The Lorenz curve for transactions associated with Alpha is computed and plotted below and interpreted in the following manner. We see that that x = 80% roughly corresponds to y = 54%, which means the 80% of the buyers of Alpha (when sorted from least to most frequent buyers) account for 54% of all the buying of Alpha in year 1. This implies the top 20% of buyers (in terms of purchase frequency) account for 46% of total purchases. The “rule” of 80/20 does not hold here; rather, it is 46/20.

In [67]:
# We are only interested in those panellists that made at least one purchase of Alpha in year 1.
# Alpha Brand Transaction Distribution of Transactions Greater Than 0
alpha_trans_dist = (
    trans_summary('Alpha', grocery_lf, 1)
    .filter(pl.col('# of Purchases') > 0)
    .group_by('# of Purchases')
    # Frequency of the Grouped Transaction Count
    .agg(pl.col('# of Purchases').count().alias('Frequency'))
    .sort('# of Purchases')
    # Add a '% of Buyers' column
    .with_columns((pl.col('Frequency')/pl.col('Frequency').sum()).alias('% of Buyers'))
    # Cumulative % of Buyers
    .with_columns((pl.col('% of Buyers').cum_sum()).alias('Cum % of Buyers'))
    # Total number of transactions made by those customers who made one purchase of Alpha
    # Total Transactions = frequency x # of transactions
    .with_columns((pl.col('Frequency')*pl.col('# of Purchases')).alias('Total Transactions'))
    # Add a '% of Transactions' column
    .with_columns((pl.col('Total Transactions')/pl.col('Total Transactions').sum()).alias('% Transactions'))
    # Cumulative % of Transactions
    .with_columns((pl.col('% Transactions').cum_sum()).alias('Cum % Transactions'))
)

(
    GT(alpha_trans_dist.collect(), rowname_col='# of Purchases')
    .tab_header(title='Alpha Buyers & Transaction Distribution',
                subtitle= 'Lorenz Curve Table')
    .tab_stubhead(label="# of Purchases")
    .fmt_percent(columns=['% of Buyers', 'Cum % of Buyers', '% Transactions', 'Cum % Transactions'], decimals=1)
    .fmt_integer(columns=['Frequency', 'Total Transactions'])
)

GT(_tbl_data=shape: (18, 7)
┌───────────┬───────────┬─────────────┬──────────┬────────────────┬────────────────┬───────────────┐
│ # of      ┆ Frequency ┆ % of Buyers ┆ Cum % of ┆ Total          ┆ % Transactions ┆ Cum %         │
│ Purchases ┆ ---       ┆ ---         ┆ Buyers   ┆ Transactions   ┆ ---            ┆ Transactions  │
│ ---       ┆ u32       ┆ f64         ┆ ---      ┆ ---            ┆ f64            ┆ ---           │
│ u32       ┆           ┆             ┆ f64      ┆ u32            ┆                ┆ f64           │
╞═══════════╪═══════════╪═════════════╪══════════╪════════════════╪════════════════╪═══════════════╡
│ 1         ┆ 733       ┆ 0.279345    ┆ 0.279345 ┆ 733            ┆ 0.080905       ┆ 0.080905      │
│ 2         ┆ 517       ┆ 0.197027    ┆ 0.476372 ┆ 1034           ┆ 0.114128       ┆ 0.195033      │
│ 3         ┆ 400       ┆ 0.152439    ┆ 0.628811 ┆ 1200           ┆ 0.13245        ┆ 0.327483      │
│ 4         ┆ 277       ┆ 0.105564    ┆ 0.734375 ┆ 1108           ┆ 0.122296       ┆ 0.449779      │
│ 5         ┆ 227       ┆ 0.086509    ┆ 0.820884 ┆ 1135           ┆ 0.125276       ┆ 0.575055      │
│ …         ┆ …         ┆ …           ┆ …        ┆ …              ┆ …              ┆ …             │
│ 14        ┆ 12        ┆ 0.004573    ┆ 0.995808 ┆ 168            ┆ 0.018543       ┆ 0.980574      │
│ 15        ┆ 6         ┆ 0.002287    ┆ 0.998095 ┆ 90             ┆ 0.009934       ┆ 0.990508      │
│ 16        ┆ 3         ┆ 0.001143    ┆ 0.999238 ┆ 48             ┆ 0.005298       ┆ 0.995806      │
│ 18        ┆ 1         ┆ 0.000381    ┆ 0.999619 ┆ 18             ┆ 0.001987       ┆ 0.997792      │
│ 20        ┆ 1         ┆ 0.000381    ┆ 1.0      ┆ 20             ┆ 0.002208       ┆ 1.0           │
└───────────┴───────────┴─────────────┴──────────┴────────────────┴────────────────┴───────────────┘, _body=<great_tables._gt_data.Body object at 0x11b55a9e0>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='center', column_width=None), ColInfo(var='Frequency', type=<ColInfoTypeEnum.default: 1>, column_label='Frequency', column_align='center', column_width=None), ColInfo(var='% of Buyers', type=<ColInfoTypeEnum.default: 1>, column_label='% of Buyers', column_align='right', column_width=None), ColInfo(var='Cum % of Buyers', type=<ColInfoTypeEnum.default: 1>, column_label='Cum % of Buyers', column_align='right', column_width=None), ColInfo(var='Total Transactions', type=<ColInfoTypeEnum.default: 1>, column_label='Total Transactions', column_align='center', column_width=None), ColInfo(var='% Transactions', type=<ColInfoTypeEnum.default: 1>, column_label='% Transactions', column_align='right', column_width=None), ColInfo(var='Cum % Transactions', type=<ColInfoTypeEnum.default: 1>, column_label='Cum % Transactions', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aab3cd0>, _spanners=Spanners([]), _heading=Heading(title='Alpha Buyers & Transaction Distribution', subtitle='Lorenz Curve Table', preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x11aab3e30>, _formats=[<great_tables._gt_data.FormatInfo object at 0x11b546740>, <great_tables._gt_data.FormatInfo object at 0x11aad1cd0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=O

- We are only interested in those panellists that made at least one purchase of Alpha in year 1.
- We compute the total number of transactions made by those customers who made one purchase of Alpha: this is obviously the number of transactions times the frequency of the transactions. We see that the 517 households that purchased Alpha twice made a total of 1034 transactions.
- What percentage of the total number of buyers of Alpha are the 733 households that made one purchase? The 517 households that made two purchases? What percentage of total purchasing is the 733 purchases made by those households that made one purchase? The 1034 purchases by made by those households that made two purchases? We first compute the total number of Alpha buyers and the total amount of purchasing by this group, and then compute the associated percentages. We see that **28% of Alpha buyers made just one purchase and their purchasing accounted for 8% of all Alpha purchasing**. Similarly, we see that **20% of buyers made two purchases and their purchasing accounted for 11% of all Alpha purchasing**.
- We compute the cumulative percentage of both `% of buyers` and `% of transactions` quantities We see, for example, that 48% of all buyers made two or fewer purchases and accounted for 20% of all the purchases of Alpha.

In [68]:
zero_row = pl.DataFrame({
    'Cum % of Buyers': [0.0],
    'Cum % Transactions': [0.0]
})

alpha_lorenz_plot = alpha_trans_dist.select('Cum % of Buyers' ,'Cum % Transactions').collect()
alpha_lorenz_plot_zero = pl.concat([zero_row, alpha_lorenz_plot])

chart = alt.Chart(alpha_lorenz_plot_zero).mark_line().encode(
    x=alt.X('Cum % of Buyers:Q', title='Cumulative % Alpha Buyers', axis=alt.Axis(format='.0%')),
    y=alt.Y('Cum % Transactions:Q', 
            title='Cumulative % Alpha Transactions', 
            axis=alt.Axis(format='.0%'),
            scale=alt.Scale(domain=[0, 1])),
).properties(
    title="Lorenz Curve for Alpha Transactions",
    width=550,
    height=450
)

chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


As noted above, we see that x = 80% roughly corresponds to y = 54%. We can compute the exact number by interpolation (linear) in the following manner:
$$\frac{y-y_{0}}{x-x_{0}}=\frac{y_{1}-y_{0}}{x_{1}-x_{0}}$$
$$(y-y_{0})=\frac{(y_{1}-y_{0})}{(x_{1}-x_{0})}(x-x_{0})$$

<div style="max-width:200px;margin-left: auto; margin-right: auto;">
<img src="references/linear_interp1.png" width="200"/>
</div>

<div style="max-width:200px;margin-left: auto; margin-right: auto;">
<img src="references/linear_interp.png" width="200"/>
</div>

- We see that 73.4% ($x_{0}$) of buyers accounted for 45.0% ($y_{0}$) of purchases, and that 82.1% ($x_{1}$) of buyers accounted for 57.5% ($y_{1}$) of purchases. Therefore, when $x$ = 80%, $y$ can be solved in the following way:
- $$0.45 + (0.8 - 0.734)\frac{(0.575 - 0.45)}{(0.821 - 0.734)} = 0.5448$$
- we see that 80% falls 76% of the way between 73.4% and 82.1%. Therefore, the associated percentage of total purchases will lie 76% of the way between 45.0% and 57.5%.
- As the “bottom” 80% account for 54% of total purchasing, it follows that the “top” 20% account for 46% of total purchasing.

In [69]:
# Numpy linear interpolation function simplifies the computation, return the interpolated y value
np.interp(0.8, xp=alpha_lorenz_plot['Cum % of Buyers'], fp=alpha_lorenz_plot['Cum % Transactions'])

np.float64(0.5448123620309051)

A related quantity of interest is the percentage of buyers that account for half of total purchasing. This can be read off the Lorenz curve in the following manner.

- We see that 73.4% of purchasers account for 45.0% of total purchases, and that 82.1% of purchasers account for 57.5% of total purchases.
- We can apply the same linear interpolation function but instead of solving for the $y$, we solve for $x$.
- $$(x-x_{0})=\frac{(x_{1}-x_{0})}{(y_{1}-y_{0})}(y-y_{0})$$
- $$\frac{(0.821-0.734)}{(0.575-0.45)}(0.5-0.45) + 0.734 = 0.769$$
- We see that 50% lies 40% of the way between 44.0% and 57.5%. Therefore, the associated percentage of purchasers lies 40% of the way between 73.4% and 82.1%, we see this is 76.9%.
- As the “bottom” 77% of buyers account for half of total purchasing, it follows that the “top” 23% also account for half of total purchasing, which we can write as 50/23. This quantity is easy for most people grasp.

In [70]:
# Reversing the axis returns the interpolated x value
np.interp(0.5, xp=alpha_lorenz_plot['Cum % Transactions'], fp=alpha_lorenz_plot['Cum % of Buyers'])

np.float64(0.7690548780487805)

##### Lorenz Curves - Spend - Continuous

Purchase frequency is a discrete quantity and we created the Lorenz curve off the distribution of transactions. We now consider how to create a Lorenz curve when the quantity of interest is continuous. We will focus on creating the Lorenz curve for the spend associated with Alpha.

In [71]:
alpha_spend_dist = (
    spend_summary('Alpha', grocery_lf, 1)
    .filter(pl.col('spend') > 0)
    # Sort Alpha customer from lowest to highest
    .sort(pl.col('spend'), descending=False)
    # Compute spend as a % of total Alpha brand spend in year 1
    # percentage of total Alpha purchasing accounted for by each panellist
    .with_columns((pl.col('spend')/pl.col('spend').sum()).alias('% Spend'))
    # Compute the cumulative percentage of spend numbers
    .with_columns((pl.col('% Spend').cum_sum()).cast(pl.Float64).alias('Cum % Spend'))
).collect()

# Compute the cumulative percentage of buyers number
spend_count = alpha_spend_dist.height
alpha_spend_dist = alpha_spend_dist.with_columns(
    pl.lit(np.linspace(1 / spend_count, 1, spend_count)).alias('Cum % Cust')
)

alpha_spend_dist

panel_id,spend,% Spend,Cum % Spend,Cum % Cust
u32,f32,f32,f64,f64
3126681,1.57,0.000047,0.000047,0.000381
3119489,1.57,0.000047,0.000094,0.000762
3117751,1.57,0.000047,0.00014,0.001143
3126803,1.57,0.000047,0.000187,0.001524
3123698,1.57,0.000047,0.000234,0.001905
…,…,…,…,…
3105099,80.5,0.002398,0.988765,0.998476
3111813,84.020004,0.002503,0.991268,0.998857
3107289,92.029999,0.002741,0.994009,0.999238


In [72]:
zero_row = pl.DataFrame({
    'Cum % Spend': [0.0],
    'Cum % Cust': [0.0]
})

alpha_lorenz_plot = alpha_spend_dist.select('Cum % Spend' ,'Cum % Cust')
alpha_lorenz_plot_zero = pl.concat([zero_row, alpha_lorenz_plot])

chart = alt.Chart(alpha_lorenz_plot_zero).mark_line().encode(
    x=alt.X('Cum % Cust:Q', title='Cumulative % Alpha Buyers', axis=alt.Axis(format='.0%')),
    y=alt.Y('Cum % Spend:Q', 
            title='Cumulative % Alpha Revenue', 
            axis=alt.Axis(format='.0%'),
            scale=alt.Scale(domain=[0, 1])),
).properties(
    title="Lorenz Curve for Alpha Spend",
    width=550,
    height=450
)

chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


Looking at `alpha_spend_dist`, we see that the "bottom" 80% of Alpha buyers account for 50.8% of its revenue in year 1, which is equivalent to 49/20. This is higher than the 46/20 we observed for transactions.

In [73]:
np.interp(0.8, xp=alpha_lorenz_plot['Cum % Cust'], fp=alpha_lorenz_plot['Cum % Spend'])

np.float64(0.5083526968955995)

#### Garbage Collect

In [74]:
exceptions = ['grocery_lf', 'sku_lf', 'kiwi_lf', 'In', 'exceptions', 'active_variables',
              'penetration', 'ppb', 'panel_size']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (pl.LazyFrame, pl.DataFrame, pl.Series, alt.Chart, alt.LayerChart, list, int, float, str, np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var

gc.collect()

114

### <span style="color:#956bbf">Exploring Multibrand Buying Behaviour</span>
---

#### Introduction

So far, we have explored purchasing at the level of the individual brand or overall category. We now step back and consider panellists’ purchasing of multiple brands (in a given time period).

To set the scene, let us first determine the percentage of category buyers that bought 1, 2, 3, 4, or 5 different brands in the course of year 1. (Note that we only have five brands in the dataset, with Other being an aggregation of several very small share brands).

In [75]:
brand_purchases = (
    trans_summary('Category', grocery_lf, 1)
    .group_by('Brands Purchased')
    .agg(pl.col('panel_id').count().alias('Category Buyer'))
    .with_columns((pl.col('Category Buyer') / pl.col('Category Buyer').sum()).alias('% Category Buyers'))
    .sort('Brands Purchased')
 ).collect()

(
    GT(brand_purchases, rowname_col='Brands Purchased')
    .tab_header(title='Category Buyers That Bought Different Brands',
                subtitle='Distribution of the number of separate brands purchased by category buyers in year 1')
    .tab_stubhead(label="Brands Purchased")
    .fmt_integer(columns=['Category Buyer'])
    .fmt_percent(columns=['% Category Buyers'], decimals=0)
    .data_color(
        columns=['% Category Buyers'],
        domain=[0, 0.7],
        palette=['white', 'rebeccapurple']
    ) 
)

GT(_tbl_data=shape: (4, 3)
┌──────────────────┬────────────────┬───────────────────┐
│ Brands Purchased ┆ Category Buyer ┆ % Category Buyers │
│ ---              ┆ ---            ┆ ---               │
│ u32              ┆ u32            ┆ f64               │
╞══════════════════╪════════════════╪═══════════════════╡
│ 1                ┆ 2964           ┆ 0.64801           │
│ 2                ┆ 1278           ┆ 0.279405          │
│ 3                ┆ 293            ┆ 0.064058          │
│ 4                ┆ 39             ┆ 0.008526          │
└──────────────────┴────────────────┴───────────────────┘, _body=<great_tables._gt_data.Body object at 0x11b55bcf0>, _boxhead=Boxhead([ColInfo(var='Brands Purchased', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands Purchased', column_align='center', column_width=None), ColInfo(var='Category Buyer', type=<ColInfoTypeEnum.default: 1>, column_label='Category Buyer', column_align='center', column_width=None), ColInfo(var='% Category Buyers', type=<ColInfoTypeEnum.default: 1>, column_label='% Category Buyers', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa43e30>, _spanners=Spanners([]), _heading=Heading(title='Category Buyers That Bought Different Brands', subtitle='Distribution of the number of separate brands purchased by category buyers in year 1', preheader=None), _stubhead='Brands Purchased', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='% Category Buyers', rows=[0]), grpname=None, colname='% Category Buyers', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7142a1')]), StyleInfo(locname=LocBody(columns='% Category Buyers', rows=[1]), grpname=None, colname='% Category Buyers', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c2aed6')]), StyleInfo(locname=LocBody(columns='% Category Buyers', rows=[2]), grpname=None, colname='% Category Buyers', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f1ecf6')]), StyleInfo(locname=LocBody(columns='% Category Buyers', rows=[3]), grpname=None, colname='% Category Buyers', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fdfdfe')])], _locale=<great_tables._gt_data.Locale object at 0x11aa40cd0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x11aad02f0>, <great_tables._gt_data.FormatInfo object at 0x11aad2c90>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True

We note that over two-thirds of the category buyers only ever bought one brand. This is despite the fact that 86% of category buyers made two or more category purchases in the course of the year. No household purchased all five brands that year.

Note: We know from analyzing the distribution of category purchase frequency that 8.9% of the panel made zero category purchases and 13.2% made one category purchase. 1 − 0.132/(1 − 0.089) = 0.855, which means 86% of year 1 category buyers made more than one category purchase that year.

Can also be computed as: (1 - 0.132 - 0.089)/(1 - 0.089) [% of panellists that more than 1 purchase] / [% of category buyers]

> How does the number of different brands purchased in the year vary as a function of the number of category purchases made during the year?

In [76]:
# number of category purchases made by each household and number of brands purchased by each household
brand_loyalty = (
    trans_summary('Category', grocery_lf, 1)
    .group_by('# of Purchases', 'Brands Purchased')
    .agg(pl.col('panel_id').count().alias('Count'))
    .collect()
    .pivot(
        on='Brands Purchased',
        index='# of Purchases',
        values='Count'
    ).sort(pl.col('# of Purchases'))
    .with_columns(pl.col('*'))
    .select('# of Purchases', '1', '2', '3', '4')
)

(
    GT(brand_loyalty, rowname_col='# of Purchases')
    .tab_header(title='# of Purchase Occasions Vs. Brands Purchased ',
                subtitle='Brands purchased as function of the number of category purchases made in year 1')
    .tab_stubhead(label="# of Purchases")
    .sub_missing(missing_text="")
    .fmt_integer()
    .data_color(
        domain=[0, 700],
        palette=['white', 'rebeccapurple'],
        na_color='white'
    ) 
)

GT(_tbl_data=shape: (23, 5)
┌────────────────┬──────┬──────┬──────┬──────┐
│ # of Purchases ┆ 1    ┆ 2    ┆ 3    ┆ 4    │
│ ---            ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ u32            ┆ u32  ┆ u32  ┆ u32  ┆ u32  │
╞════════════════╪══════╪══════╪══════╪══════╡
│ 1              ┆ 655  ┆ 12   ┆ null ┆ null │
│ 2              ┆ 573  ┆ 184  ┆ 2    ┆ null │
│ 3              ┆ 516  ┆ 216  ┆ 24   ┆ 1    │
│ 4              ┆ 370  ┆ 193  ┆ 47   ┆ 2    │
│ 5              ┆ 278  ┆ 189  ┆ 43   ┆ 4    │
│ …              ┆ …    ┆ …    ┆ …    ┆ …    │
│ 19             ┆ null ┆ 2    ┆ null ┆ null │
│ 20             ┆ 2    ┆ 1    ┆ 1    ┆ null │
│ 22             ┆ null ┆ 1    ┆ null ┆ null │
│ 25             ┆ null ┆ 1    ┆ null ┆ null │
│ 27             ┆ null ┆ null ┆ null ┆ 1    │
└────────────────┴──────┴──────┴──────┴──────┘, _body=<great_tables._gt_data.Body object at 0x11aa7b2a0>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11aa43a70>, _spanners=Spanners([]), _heading=Heading(title='# of Purchase Occasions Vs. Brands Purchased ', subtitle='Brands purchased as function of the number of category purchases made in year 1', preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='# of Purchases', rows=[0]), grpname=None, colname='# of Purchases', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffffff')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[1]), grpname=None, colname='# of Purchases', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fffeff')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[2]), grpname=None, colname='# of Purchases', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fefeff')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[3]), grpname=None, colname='# of Purchases', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fefefe')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[4]), grpname=None, colname='# of Purchases', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fefefe')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[5]), grpname=None, colname='# of Purchases', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fefdfe')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[6]), grpname=None, colname='# of Purchases', rownum=6, colnum=None, styles=[CellStyleText(color='#000000'

Looking at the resulting table output, we see that there is quite a high level of sole-brand loyalty (i.e., only buying one brand), even as the number of category purchases increases. We have two panellists that made 20 category purchases, all with the same brand. That’s some level of
loyalty!

We can compute the (weighted) average number of different brands purchased for each level of category purchasing.

In [77]:
wa_brands_purchased = (
    trans_summary('Category', grocery_lf, 1)
    .group_by('# of Purchases', 'Brands Purchased')
    .agg(pl.col('panel_id').count().alias('Count'))
    .with_columns((pl.col('Brands Purchased') * pl.col('Count')).alias('Product'))
    .group_by('# of Purchases')
    .agg(
        (pl.col('Product').sum() / pl.col('Count').sum()) .alias('Weighted Average')
    ).sort(by='# of Purchases')
).collect()

(
    GT(wa_brands_purchased, rowname_col='# of Purchases')
    .tab_header(title='W.A. # of Different Brands Purchased')
    .tab_stubhead(label="# of Purchases")
    .fmt_number()
    .data_color(
        domain=[1, 4],
        palette=['white', 'rebeccapurple']
    )
    .cols_label({'Weighted Average': 'Weighted Average Brands'}) 
)

GT(_tbl_data=shape: (23, 2)
┌────────────────┬──────────────────┐
│ # of Purchases ┆ Weighted Average │
│ ---            ┆ ---              │
│ u32            ┆ f64              │
╞════════════════╪══════════════════╡
│ 1              ┆ 1.017991         │
│ 2              ┆ 1.247694         │
│ 3              ┆ 1.352708         │
│ 4              ┆ 1.478758         │
│ 5              ┆ 1.558366         │
│ …              ┆ …                │
│ 19             ┆ 2.0              │
│ 20             ┆ 1.75             │
│ 22             ┆ 2.0              │
│ 25             ┆ 2.0              │
│ 27             ┆ 4.0              │
└────────────────┴──────────────────┘, _body=<great_tables._gt_data.Body object at 0x11b52eb20>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='center', column_width=None), ColInfo(var='Weighted Average', type=<ColInfoTypeEnum.default: 1>, column_label='Weighted Average Brands', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x1086023c0>, _spanners=Spanners([]), _heading=Heading(title='W.A. # of Different Brands Purchased', subtitle=None, preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='# of Purchases', rows=[0]), grpname=None, colname='# of Purchases', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffffff')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[1]), grpname=None, colname='# of Purchases', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ccbbdd')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[2]), grpname=None, colname='# of Purchases', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9977bb')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[3]), grpname=None, colname='# of Purchases', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[4]), grpname=None, colname='# of Purchases', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[5]), grpname=None, colname='# of Purchases', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[6]), grpname=None, colname='# of Purchases', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='# of Purchases', rows=[7]), grpname=None, colname='# of Purchases', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='

We note that the average number of brands purchased does increase as a function of category purchasing. This should not come as a surprise, as more category purchases equals more opportunities to buy different brands. We saw in earlier that two brands (Alpha and Bravo) had a combined value market share of 86%. As such, the relatively low number of different brands purchased in the year is not too surprising.

We now consider three common analyses designed to give insight into the nature of multibrand buying behaviour.

#### Duplication of Purchase

We know from our analysis of penetration and PPB earlier that 52% of households purchased Alpha and 51% of households purchased Bravo. What percentage of Alpha buyers also purchased Bravo during the year (and vice-versa)? The answer to these (and similar) questions is provided by a duplication of purchase analysis.

Before performing this analysis on our dataset, let us consider a toy example. The following table summarizes the purchasing of four brands by six households.

|      | A | B | C | D |
|------|---|---|---|---|
| HH01 | **1** | 0 | **2** | 0 |
| HH02 | 0 | 1 | 0 | 0 |
| HH03 | **1** | **3** | 0 | 0 |
| HH04 | 0 | 0 | 1 | 4 |
| HH05 | **1** | **1** | 0 | **1** |
| HH06 | 0 | 0 | 0 | 1 |

We see that three households made at least one purchase of brand A, three households made at least one purchase of brand B, and so on. How many brand A buyers also purchased brand B? Two (HH03 and HH05). How many brand A buyers also purchased brand C? One (HH01). Repeating this for all brands gives us the following table, which we will call a **duplication count table**.

|   | A | B | C | D |
|---|---|---|---|---|
| **A** | 3 | 2 | 1 | 1 |
| **B** | 2 | 3 | 0 | 1 |
| **C** | 1 | 0 | 2 | 1 |
| **D** | 1 | 1 | 1 | 3 |

For any given row, the number in each cell is the number of buyers of the brand associated with that row that also purchased the brand associated with that column. (The diagonal is obviously the number of buyers of each brand.) Looking at the row for brand A, we see that three households
purchased that brand. Two of these households (67%) also made at least one purchase of brand B, and one of these three households (33%) also made at least one purchase of brand C. These row percentages are reported in the following table, which we call the **duplication of purchase table**. (By convention, we leave the diagonal blank.)

|   | A   | B   | C   | D   |
|:---:|:---:|:---:|:---:|:---:|
| **A** |     | 67% | 33% | 33% |
| **B** | 67% |     | 0%  | 33% |
| **C** | 50% | 0%  |     | 50% |
| **D** | 33% | 33% | 33% |     |

How can we create this table efficiently when we have a large number of panellists? One approach, which makes use of matrix multiplication, is as follows:
- We create that we will call an “**ever buyers**” matrix, which is a matrix of size (number of panellists) × (number of brands), where each cell takes on a value of 1 if the panellist (row) ever purchased the brand (column) in the period of interest; 0 otherwise.

|      | A | B | C | D |
|------|---|---|---|---|
| HH01 | 1 | 0 | 1 | 0 |
| HH02 | 0 | 1 | 0 | 0 |
| HH03 | 1 | 1 | 0 | 0 |
| HH04 | 0 | 0 | 1 | 1 |
| HH05 | 1 | 1 | 0 | 1 |
| HH06 | 0 | 0 | 0 | 1 |

- Pre-multiplying the “ever buyers” matrix by its own transpose gives us the **duplication count table** we created above $A^{T}\cdot A=B$. (Evaluate the example below.)
  
Transposed "ever buyer" matrix:

|   | HH01 | HH02 | HH03 | HH04 | HH05 | HH06 |
|:---:|:----:|:----:|:----:|:----:|:----:|:----:|
| A | 1    | 0    | 1    | 0    | 1    | 0    |
| B | 0    | 1    | 1    | 0    | 1    | 0    |
| C | 1    | 0    | 0    | 1    | 0    | 0    |
| D | 0    | 0    | 0    | 1    | 1    | 1    |

Matrix Multiply By:
|      | A | B | C | D |
|------|---|---|---|---|
| HH01 | 1 | 0 | 1 | 0 |
| HH02 | 0 | 1 | 0 | 0 |
| HH03 | 1 | 1 | 0 | 0 |
| HH04 | 0 | 0 | 1 | 1 |
| HH05 | 1 | 1 | 0 | 1 |
| HH06 | 0 | 0 | 0 | 1 |

- The 'ever buyer' matrix effectively creates a binary of 1 (Yes) or 0 (No) to factor in the required inclusions or exclusions of intersection between buyers of different brands. In this case, as we multiply the binary matrix and its transposed form together, we are computing a matrix that records the count of existing intersection of buyers of a brand who also purchased other brands. 
- Dividing each cell by the number of buyers of the brand associated with each row gives us the **duplication of purchase table**.

[Matrix Multiplication Visualizer](http://matrixmultiplication.xyz)

In [78]:
# m x n matrix where m = # of panellists, n = # of brands and value is number of purchases
brand_purch = np.array([
    [1, 0, 2, 0],
    [0, 1, 0, 0],
    [1, 3, 0, 0],
    [0, 0, 1, 4],
    [1, 1, 0, 1],
    [0, 0, 0, 1]
])

# m x n matrix where value is 1 if buyer ever purchased, 0 if never purchased
ever_buyers = np.where(brand_purch != 0, 1, 0)

# n x n square matrix representing count of row/column brand buyers who also purchased column/row brands
dup_count = ever_buyers.T @ ever_buyers
print('Duplication Count Matrix:')
print(dup_count, dup_count.shape)

print()
# Diagonal vector of dup_count matrix representing the number of buyers of each ROW brand
brand_buyers = np.diag(dup_count).reshape(4, 1)
print('Number of Buyers of Each Brand:')
print(brand_buyers, brand_buyers.shape)

# n x n square matrix representing the proportion of brand buyers for each ROW brand that also purchased other COLUMN brands
dup_purchase = dup_count / brand_buyers
print('Duplication of Purchase Matrix:')
print(dup_purchase, dup_purchase.shape)

Duplication Count Matrix:
[[3 2 1 1]
 [2 3 0 1]
 [1 0 2 1]
 [1 1 1 3]] (4, 4)

Number of Buyers of Each Brand:
[[3]
 [3]
 [2]
 [3]] (4, 1)
Duplication of Purchase Matrix:
[[1.         0.66666667 0.33333333 0.33333333]
 [0.66666667 1.         0.         0.33333333]
 [0.5        0.         1.         0.5       ]
 [0.33333333 0.33333333 0.33333333 1.        ]] (4, 4)


In [79]:
brand_purchases = trans_pivot(grocery_lf, 1).select('Alpha', 'Bravo', 'Charlie', 'Delta', 'Other')

# Step 1: Create ever buyers matrix
ever_buyers = brand_purchases.select(
    [pl.when(pl.col(col) > 0).then(1).otherwise(0).alias(col) for col in brand_purchases.columns]
).to_numpy()

# Step 2: Compute duplication count matrix
dup_count = ever_buyers.T @ ever_buyers
brand_buyers = np.diag(dup_count).reshape(5, 1)

# Step 3: Compute duplication of purchase matrix
dup_purchase = dup_count / brand_buyers

# DataFrame conversion for tabular view
dup_count_df = (
    pl.from_numpy(dup_count, schema=brand_purchases.columns)
    .with_columns(pl.Series('Brands', brand_purchases.columns))
)

dup_purchase_df = (
    pl.from_numpy(dup_purchase, schema=brand_purchases.columns)
    .with_columns(pl.Series('Brands', brand_purchases.columns))
)

In [80]:
(
    GT(dup_count_df, rowname_col='Brands')
    .tab_header(title='Duplication Count Matrix',
                subtitle='Count of buyers who purchased row AND column brands')
    .tab_stubhead(label="Brands")
    .sub_missing(missing_text="")
    .fmt_integer()
    .data_color(
        domain=[0, 900],
        palette=['white', 'rebeccapurple'],
    ) 
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('lightgrey'),
            
        ],
        locations=[
            loc.body(['Alpha'],['Alpha']), 
            loc.body(['Bravo'],['Bravo']),
            loc.body(['Charlie'],['Charlie']),
            loc.body(['Delta'],['Delta']),
            loc.body(['Other'],['Other'])
        ]
    )
    .tab_source_note(md("""**Read Row/Column as**: _# of People who Purchased [Brand] / Who also Purchased [Brand]_"""))
)

GT(_tbl_data=shape: (5, 6)
┌───────┬───────┬─────────┬───────┬───────┬─────────┐
│ Alpha ┆ Bravo ┆ Charlie ┆ Delta ┆ Other ┆ Brands  │
│ ---   ┆ ---   ┆ ---     ┆ ---   ┆ ---   ┆ ---     │
│ i32   ┆ i32   ┆ i32     ┆ i32   ┆ i32   ┆ str     │
╞═══════╪═══════╪═════════╪═══════╪═══════╪═════════╡
│ 2624  ┆ 896   ┆ 403     ┆ 239   ┆ 69    ┆ Alpha   │
│ 896   ┆ 2562  ┆ 382     ┆ 140   ┆ 110   ┆ Bravo   │
│ 403   ┆ 382   ┆ 813     ┆ 116   ┆ 26    ┆ Charlie │
│ 239   ┆ 140   ┆ 116     ┆ 380   ┆ 10    ┆ Delta   │
│ 69    ┆ 110   ┆ 26      ┆ 10    ┆ 176   ┆ Other   │
└───────┴───────┴─────────┴───────┴───────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x11b558280>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Brands', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10f7e0ad0>, _spanners=Spanners([]), _heading=Heading(title='Duplication Count Matrix', subtitle='Count of buyers who purchased row AND column brands', preheader=None), _stubhead='Brands', _source_notes=[Md(text='**Read Row/Column as**: _# of People who Purchased [Brand] / Who also Purchased [Brand]_')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#673499')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#baa4d1')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d6c9e4')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f3eff7')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#673499')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(colo

In [81]:
(
    GT(dup_purchase_df, rowname_col='Brands')
    .tab_header(title='Duplication of Purchase Matrix',
                subtitle='% of brand buyers that also purchased other brands')
    .tab_stubhead(label="Brands")
    .sub_missing(missing_text="")
    .fmt_percent(decimals=0)
    .data_color(
        domain=[0, 0.7],
        palette=['white', 'rebeccapurple'],
    ) 
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('white'),
            
        ],
        locations=[
            loc.body(0,0),
            loc.body(1,1),
            loc.body(2,2),
            loc.body(3,3),
            loc.body(4,4)
        ]
    )
    .tab_source_note(md("""**Read Row/Column as**: _% of People who Purchased [Brand] / Who also Purchased [Brand]_"""))
)

GT(_tbl_data=shape: (5, 6)
┌──────────┬──────────┬──────────┬──────────┬──────────┬─────────┐
│ Alpha    ┆ Bravo    ┆ Charlie  ┆ Delta    ┆ Other    ┆ Brands  │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ str     │
╞══════════╪══════════╪══════════╪══════════╪══════════╪═════════╡
│ 1.0      ┆ 0.341463 ┆ 0.153582 ┆ 0.091082 ┆ 0.026296 ┆ Alpha   │
│ 0.349727 ┆ 1.0      ┆ 0.149102 ┆ 0.054645 ┆ 0.042935 ┆ Bravo   │
│ 0.495695 ┆ 0.469865 ┆ 1.0      ┆ 0.142681 ┆ 0.03198  ┆ Charlie │
│ 0.628947 ┆ 0.368421 ┆ 0.305263 ┆ 1.0      ┆ 0.026316 ┆ Delta   │
│ 0.392045 ┆ 0.625    ┆ 0.147727 ┆ 0.056818 ┆ 1.0      ┆ Other   │
└──────────┴──────────┴──────────┴──────────┴──────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x11b55bc00>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Brands', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b5d72d0>, _spanners=Spanners([]), _heading=Heading(title='Duplication of Purchase Matrix', subtitle='% of brand buyers that also purchased other brands', preheader=None), _stubhead='Brands', _source_notes=[Md(text='**Read Row/Column as**: _% of People who Purchased [Brand] / Who also Purchased [Brand]_')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b399cc')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#936fb7')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7648a3')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#a98dc6')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b49bcd')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=N

The duplication of purchase matrix for year 1 is computed above. We see that 34% of those panellists that purchased Alpha in year 1 also made at least one purchase of Bravo that year. We see that 15% of Alpha buyers also bought Charlie, whereas 50% of Charlie buyers also bought Alpha. This asymmetry is not surprising given the relative size of the two brands.

#### Share of Category Requirements (SCR)

The duplication of purchase table tells use that 34% of Alpha buyers also purchased Bravo, 15% also purchased Charlie, and so on. This “polygamous purchasing” leads to an obvious question: How “loyal” are they to Alpha? The answer to this question obviously depends on what we mean by “loyal”. One commonly used measure of loyalty is **share of category requirements (SCR)**, which is the percentage of category volume that the brand represents among its buyers.

The SCR numbers for each brand are reported below. We see that Alpha has a share of category requirements of 69%. This means that 69% of the *total category volume purchased by buyers of Alpha goes to Alpha*. Contrast this to Delta, which has an SCR of 40%. This means that 40% of the volume purchased in the category by buyers of Delta goes to that brand.

In [82]:
# List of brands
brands = ['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other']

# Initialize results list
scr_results = []

with pl.StringCache():
    # Compute total category volume
    total_vol = vol_summary('Category', grocery_lf, 1)

    for brand in brands:
        # Join brand-specific volume to total category volume to get total category volume for panelists who purchased the brand
        scr_lf = (
            total_vol.join(
                other=vol_summary(brand, grocery_lf, 1),
                on='panel_id',
                how='inner'
            )
            .with_columns(
                # Compute total brand and category volumes
                # Total purchasing of each brand
                (pl.col('volume_right').sum()).alias('Brand Volume'),
                # The total amount of category purchasing conditioned on the fact that at least one purchase of the brand of interest was made.
                (pl.col('volume').sum()).alias('Category Volume'),          
            )
            .with_columns(
                # Compute SCR - ratio of brand purchasing to category purchasing
                (pl.col('Brand Volume').sum() / pl.col('Category Volume').sum()).alias('SCR')          
            )
            .select('Brand Volume', 'Category Volume', 'SCR')
            .collect()
        )
        
        # Append results
        scr_results.append({
            "Brands": brand,
            "Brand Purchasing": scr_lf.item(1, "Brand Volume"),
            "Category Purchasing": scr_lf.item(1, "Category Volume"),
            "SCR": scr_lf.item(1, "SCR")
        })

# Create final DataFrame
scr = pl.DataFrame(scr_results)

In [83]:
(
    GT(scr, rowname_col='Brands')
    .tab_header(title='Share of Category Requirements (SCR)',
                subtitle='% of category volume that the brand represents among its buyers')
    .tab_stubhead(label="Brands")
    .fmt_integer(columns=['Brand Purchasing', 'Category Purchasing'])
    .fmt_percent(columns='SCR', decimals=0)
    .data_color(
        domain=[0, 0.7],
        palette=['white', 'rebeccapurple'],
        columns='SCR'
    )
)

GT(_tbl_data=shape: (5, 4)
┌─────────┬──────────────────┬─────────────────────┬──────────┐
│ Brands  ┆ Brand Purchasing ┆ Category Purchasing ┆ SCR      │
│ ---     ┆ ---              ┆ ---                 ┆ ---      │
│ str     ┆ f64              ┆ f64                 ┆ f64      │
╞═════════╪══════════════════╪═════════════════════╪══════════╡
│ Alpha   ┆ 9166.25          ┆ 13315.0             ┆ 0.688415 │
│ Bravo   ┆ 8240.35          ┆ 12121.225           ┆ 0.679828 │
│ Charlie ┆ 2171.125         ┆ 4779.625            ┆ 0.454246 │
│ Delta   ┆ 921.0            ┆ 2277.4              ┆ 0.404409 │
│ Other   ┆ 286.275          ┆ 979.8               ┆ 0.292177 │
└─────────┴──────────────────┴─────────────────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x11b5e36b0>, _boxhead=Boxhead([ColInfo(var='Brands', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands', column_align='left', column_width=None), ColInfo(var='Brand Purchasing', type=<ColInfoTypeEnum.default: 1>, column_label='Brand Purchasing', column_align='right', column_width=None), ColInfo(var='Category Purchasing', type=<ColInfoTypeEnum.default: 1>, column_label='Category Purchasing', column_align='right', column_width=None), ColInfo(var='SCR', type=<ColInfoTypeEnum.default: 1>, column_label='SCR', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b57ac10>, _spanners=Spanners([]), _heading=Heading(title='Share of Category Requirements (SCR)', subtitle='% of category volume that the brand represents among its buyers', preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='SCR', rows=[0]), grpname=None, colname='SCR', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69369b')]), StyleInfo(locname=LocBody(columns='SCR', rows=[1]), grpname=None, colname='SCR', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#6a399c')]), StyleInfo(locname=LocBody(columns='SCR', rows=[2]), grpname=None, colname='SCR', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9c7bbd')]), StyleInfo(locname=LocBody(columns='SCR', rows=[3]), grpname=None, colname='SCR', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#a789c4')]), StyleInfo(locname=LocBody(columns='SCR', rows=[4]), grpname=None, colname='SCR', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#bfaad4')])], _locale=<great_tables._gt_data.Locale object at 0x11b579710>, _formats=[<great_tables._gt_data.FormatInfo object at 0x11b54cd10>, <great_tables._gt_data.FormatInfo object at 0x11b54d0d0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', va

In [84]:
# Method 2

brand_vol = vol_pivot(grocery_lf, 1).fill_null(0)

ever_buyers = brand_vol.select(
    [pl.when(pl.col(col) > 0).then(1).otherwise(0).alias(col) for col in brand_vol.columns]
).to_numpy()

brand_vol = brand_vol.to_numpy()

cross_purchasing = ever_buyers.T @ brand_vol
brand_purchasing = np.diag(cross_purchasing)
category_purchasing = np.sum(cross_purchasing, axis=1)
scr = brand_purchasing / category_purchasing
scr

array([0.68841532, 0.67982815, 0.45424589, 0.40440854, 0.29217697])

#### Cross Purchasing

##### Cross Purchasing - *By Volume Purchase*

We have just seen that 69% of total category volume purchasing by the buyers of Alpha went to that brand. We know from the duplication of purchase analysis that 15% of Alpha buyers also purchased Charlie. How much of their category *volume* purchasing went to Charlie? This is answered via **cross purchase analysis** (sometimes called a **combination purchase analysis**).

To illustrate the logic of the associated calculations, let us revisit the toy problem introduced earlier. We have the following summary of the purchasing of four brands by six households and the associated “ever buyers” matrix. We will assume the "ever buyers" matrix reports volume purchasing in kilograms.

**Brand Purchasing**
|      | A | B | C | D | Total |
|:----:|:---:|:---:|:---:|:---:|:-----:|
| HH01 | **1** | 0 | **2** | 0 | **3**     |
| HH02 | 0 | 1 | 0 | 0 | 1     |
| HH03 | **1** | **3** | 0 | 0 | **4**     |
| HH04 | 0 | 0 | 1 | 4 | 5     |
| HH05 | **1** | **1** | 0 | **1** | **3**     |
| HH06 | 0 | 0 | 0 | 1 | 1     |

**Ever Buyers Matrix**
|      | A | B | C | D |
|------|---|---|---|---|
| HH01 | 1 | 0 | 1 | 0 |
| HH02 | 0 | 1 | 0 | 0 |
| HH03 | 1 | 1 | 0 | 0 |
| HH04 | 0 | 0 | 1 | 1 |
| HH05 | 1 | 1 | 0 | 1 |
| HH06 | 0 | 0 | 0 | 1 |

We see that buyers of brand A purchased 3 kg (1 + 1 + 1, from A column) of brand A and a total of 10 kg (3 + 4 + 3, from total column) in the category (i.e., SCR = 30%). We see that they also purchased 4 kg (3 + 1, from B column) of brand B, 2 kg (2, from C column) of brand C and 1 kg (1, from D column) of brand D. The associated numbers for all brands are given in the following table:

|   | A | B | C | D |
|---|---|---|---|---|
| **A** | 3 | 4 | 2 | 1 |
| **B** | 2 | 5 | 0 | 1 |
| **C** | 1 | 0 | 3 | 4 |
| **D** | 1 | 1 | 1 | 6 |

The sum of the elements of each row gives us the total amount of category purchasing by buyers of the brand of that row. Dividing each row entry by the sum of that row’s elements gives us the following **cross purchasing table**, the *diagonal of which is obviously SCR*.

|   | A   | B   | C   | D    |
|:---:|:---:|:---:|:---:|:----:|
| **A** | 30% | 40% | 20% | 10%  |
| **B** | 25% | 63% | 0%  | 13%  |
| **C** | 13% | 0%  | 38% | 50%  |
| **D** | 11% | 11% | 11% | 67%  |

How do we create the table efficiently when we have a large number of panellists? One approach is to pre-multiply the panellist × brand volume purchasing summary table by the transpose of the “ever buyers” matrix $A^{T}\cdot B$:

Transposed "ever buyer" matrix:

|   | HH01 | HH02 | HH03 | HH04 | HH05 | HH06 |
|:---:|:----:|:----:|:----:|:----:|:----:|:----:|
| A | 1    | 0    | 1    | 0    | 1    | 0    |
| B | 0    | 1    | 1    | 0    | 1    | 0    |
| C | 1    | 0    | 0    | 1    | 0    | 0    |
| D | 0    | 0    | 0    | 1    | 1    | 1    |

Multiplied by Brand Volume Purchasing:

|      | A | B | C | D |
|:----:|:---:|:---:|:---:|:---:|
| HH01 | 1 | 0 | 2 | 0 |
| HH02 | 0 | 1 | 0 | 0 |
| HH03 | 1 | 3 | 0 | 0 |
| HH04 | 0 | 0 | 1 | 4 |
| HH05 | 1 | 1 | 0 | 1 |
| HH06 | 0 | 0 | 0 | 1 |

The 'ever buyer' matrix creates a binary of 1 (buyer) or 0 (non-buyer) to help factor in the inclusions/exclusions and overlaps between brands purchased by all buyers. In this case, as we multiply the transposed form of the binary matrix with the quantity/volume matrix representing the quantity of purchase of different brands by each buyer, we are solving for a matrix that represents the total volume of purchasing for a set of brands by a set of buyers and their purchases of other brands.

In [85]:
# m x n matrix where m = # of panellists, n = # of brands and value is volume of purchases 
brand_purch = np.array([
    [1, 0, 2, 0],
    [0, 1, 0, 0],
    [1, 3, 0, 0],
    [0, 0, 1, 4],
    [1, 1, 0, 1],
    [0, 0, 0, 1]
])

# m x n matrix where value is 1 if buyer ever purchased, 0 if never purchased
ever_buyers = np.where(brand_purch != 0, 1, 0)

# n x n square matrix representing row brand buyers and their volume/unit purchases from row brand and column brands
cross_purch_matrix = ever_buyers.T @ brand_purch
print('Cross Purchasing Matrix:')
print(cross_purch_matrix, cross_purch_matrix.shape)

print()
# The sum of the elements of each row gives us the total amount of category purchasing (volume) by buyers of the brand of that row
category_purch = cross_purch_matrix.sum(axis=1).reshape(4, 1)
print('Total Amount of Category Purchasing by Buyers of the Row Brand:')
print(brand_buyers, brand_buyers.shape)

print()
# n x n square matrix representing the proportion of volume purchased by row brand buyers for each COLUMN brands relative to total purchased of the row brand buyer
cross_purch = cross_purch_matrix / category_purch
print('Cross Purchasing Matrix as a % of Total Volume Purchased of Each Brand:')
print(cross_purch, cross_purch.shape)

Cross Purchasing Matrix:
[[3 4 2 1]
 [2 5 0 1]
 [1 0 3 4]
 [1 1 1 6]] (4, 4)

Total Amount of Category Purchasing by Buyers of the Row Brand:
[[2624]
 [2562]
 [ 813]
 [ 380]
 [ 176]] (5, 1)

Cross Purchasing Matrix as a % of Total Volume Purchased of Each Brand:
[[0.3        0.4        0.2        0.1       ]
 [0.25       0.625      0.         0.125     ]
 [0.125      0.         0.375      0.5       ]
 [0.11111111 0.11111111 0.11111111 0.66666667]] (4, 4)


The cross purchasing analysis for year 1 is computed and reported below. We see that for those panellists that purchased Alpha at least once in year 1, 69% of their category volume purchased went to Alpha, 18% went to Bravo, 8% to Charlie, and so on.

In [86]:
brand_vol_purchases = vol_pivot(grocery_lf, 1).fill_null(0)

# Step 1: Create ever buyers matrix
ever_buyers = brand_vol_purchases.select(
    [pl.when(pl.col(col) > 0).then(1).otherwise(0).alias(col) for col in brand_purchases.columns]
).to_numpy()

# Step 2: Compute cross purchase matrix
cross_purch_matrix = ever_buyers.T @ brand_vol_purchases
total_volume_purchases = np.sum(cross_purch_matrix, axis=1).reshape(5, 1)

# Step 3: Compute cross purchase matrix as a % of total volume purchased of each brand
cross_purch = cross_purch_matrix / total_volume_purchases

# DataFrame conversion for tabular view
cross_purch_matrix_df = (
    pl.from_numpy(cross_purch_matrix, schema=brand_vol_purchases.columns)
    .with_columns(pl.Series('Brands', brand_vol_purchases.columns))
)

cross_purch_df = (
    pl.from_numpy(cross_purch, schema=brand_vol_purchases.columns)
    .with_columns(pl.Series('Brands', brand_vol_purchases.columns))
)

In [87]:
(
    GT(cross_purch_matrix_df, rowname_col='Brands')
    .tab_header(title='Cross Purchasing Analysis',
                subtitle='Total Volume of Category Purchasing by Buyers of the Brand (by row)')
    .tab_stubhead(label="Brands")
    .sub_missing(missing_text="")
    .fmt_integer()
    .data_color(
        domain=[0, 3000],
        palette=['white', 'rebeccapurple'],
    ) 
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('lightgrey'),
            
        ],
        locations=[
            loc.body(0,0),
            loc.body(1,1),
            loc.body(2,2),
            loc.body(3,3),
            loc.body(4,4)
        ]
    )
    .tab_source_note(md("""**Read Row/Column as**: _Purchasers of [Brands] / Total Volume Purchased of [Brands]_"""))
)

GT(_tbl_data=shape: (5, 6)
┌─────────┬─────────┬──────────┬────────┬─────────┬─────────┐
│ Alpha   ┆ Bravo   ┆ Charlie  ┆ Delta  ┆ Other   ┆ Brands  │
│ ---     ┆ ---     ┆ ---      ┆ ---    ┆ ---     ┆ ---     │
│ f64     ┆ f64     ┆ f64      ┆ f64    ┆ f64     ┆ str     │
╞═════════╪═════════╪══════════╪════════╪═════════╪═════════╡
│ 9166.25 ┆ 2461.4  ┆ 1076.325 ┆ 503.5  ┆ 107.525 ┆ Alpha   │
│ 2521.8  ┆ 8240.35 ┆ 907.275  ┆ 281.5  ┆ 170.3   ┆ Bravo   │
│ 1221.5  ┆ 1130.75 ┆ 2171.125 ┆ 220.25 ┆ 36.0    ┆ Charlie │
│ 700.55  ┆ 310.65  ┆ 331.925  ┆ 921.0  ┆ 13.275  ┆ Delta   │
│ 222.9   ┆ 397.5   ┆ 53.375   ┆ 19.75  ┆ 286.275 ┆ Other   │
└─────────┴─────────┴──────────┴────────┴─────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x11b551ea0>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Brands', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b57b460>, _spanners=Spanners([]), _heading=Heading(title='Cross Purchasing Analysis', subtitle='Total Volume of Category Purchasing by Buyers of the Brand (by row)', preheader=None), _stubhead='Brands', _source_notes=[Md(text='**Read Row/Column as**: _Purchasers of [Brands] / Total Volume Purchased of [Brands]_')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7e54a9')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c1acd5')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#dbcfe7')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f4f0f7')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8158ab')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, styl

In [88]:
(
    GT(cross_purch_df, rowname_col='Brands')
    .tab_header(title='Cross Purchasing Analysis',
                subtitle='% of Total Volume of Category Purchasing by Buyers of the Brand (by row)')
    .tab_stubhead(label="Brands")
    .sub_missing(missing_text="")
    .fmt_percent(decimals=0)
    .data_color(
        domain=[0, 0.7],
        palette=['white', 'rebeccapurple'],
    ) 
    .tab_source_note(md("""**Read Row/Column as**: _Purchasers of [Brands] / % Total Volume Purchased of [Brands]_"""))
)

GT(_tbl_data=shape: (5, 6)
┌──────────┬──────────┬──────────┬──────────┬──────────┬─────────┐
│ Alpha    ┆ Bravo    ┆ Charlie  ┆ Delta    ┆ Other    ┆ Brands  │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ str     │
╞══════════╪══════════╪══════════╪══════════╪══════════╪═════════╡
│ 0.688415 ┆ 0.184859 ┆ 0.080836 ┆ 0.037814 ┆ 0.008075 ┆ Alpha   │
│ 0.208048 ┆ 0.679828 ┆ 0.07485  ┆ 0.023224 ┆ 0.01405  ┆ Bravo   │
│ 0.255564 ┆ 0.236577 ┆ 0.454246 ┆ 0.046081 ┆ 0.007532 ┆ Charlie │
│ 0.30761  ┆ 0.136406 ┆ 0.145747 ┆ 0.404409 ┆ 0.005829 ┆ Delta   │
│ 0.227495 ┆ 0.405695 ┆ 0.054475 ┆ 0.020157 ┆ 0.292177 ┆ Other   │
└──────────┴──────────┴──────────┴──────────┴──────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x11b553c00>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Brands', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b530170>, _spanners=Spanners([]), _heading=Heading(title='Cross Purchasing Analysis', subtitle='% of Total Volume of Category Purchasing by Buyers of the Brand (by row)', preheader=None), _stubhead='Brands', _source_notes=[Md(text='**Read Row/Column as**: _Purchasers of [Brands] / % Total Volume Purchased of [Brands]_')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69369b')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d2c2e1')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c7b5da')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#bca5d2')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#cdbdde')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d7c9e4')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='

##### Importance of Competition Plot - *By Volume Purchase*

For a given brand, we can plot the associated row entries as a pie chart, for example, see the *Importance of competition plot*, which plots the percentage of total volume of category purchasing by buyers of Alpha.

In [89]:
cross_purch_plot = (
    pl.from_numpy(cross_purch.T, schema=brand_vol_purchases.columns, orient='row')
    .with_columns(pl.Series('Brands', brand_vol_purchases.columns))
)

base = alt.Chart(cross_purch_plot).encode(
    alt.Theta("Alpha:Q").stack(True),
    alt.Color("Brands:N")
).properties(
    width=600,
    title={'text': 'Importance of Competition to Buyers of Alpha',
           'subtitle': '% of Total Volume of Category Purchasing by Buyers of Alpha'},
)

pie = base.mark_arc(outerRadius=145)
text = base.mark_text(radius=165, size=15).encode(
    text=alt.Text("Alpha:N", format=".0%"))

pie + text

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


##### Importance Against Expectation - *By Volume Purchase*

As previously noted when we computed SCR, we see that Alpha accounts for 69% of category purchasing by the buyers of Alpha. We see from the cross purchase analysis see that Bravo accounts for 18% of their category purchasing. Is this large or small? One way of answering this question is to compare actual purchasing against expectation given general purchasing patterns in the category.

In [90]:
# Share of Category Purchasing - Alpha Brand Selected
share_of_cat_purch = cross_purch[0]

# Market Share - Volume
market_share = (
    brand_vol_purchases.sum() / # sum all columns -> returns a single row dataframe
    brand_vol_purchases.sum_horizontal().sum() # sum of total category volume sales -> retruns a scalar value
).to_numpy().reshape(5)

# Share of residual purchasing -- Alpha Buyers
# The percentage of the category purchasing not accounted for by Alpha that goes to each of the other brands
residual_purch_brand = share_of_cat_purch / (1 - share_of_cat_purch[0])

# Share of residual purchasing -- Category Buyers
# Residual share of category purchasing (across all category buyers) once Alpha is removed
# when we exclude Alpha, what percentage of (the remaining) category purchasing goes to each of the other brands)
residual_purch_category = market_share / (1 - market_share[0])

# Index against expectation
index_against_expect = 100 * residual_purch_brand/residual_purch_category

# Importance Against Expectations - Create DataFrame
stack = np.vstack((share_of_cat_purch, 
                   market_share, 
                   residual_purch_brand, 
                   residual_purch_category, 
                   index_against_expect))

cols = pl.DataFrame({'Col': ['Share of Category Purchasing', 
        'Volume Market Share', 
        'Share of Residual Purchasing - Alpha Buyers', 
        'Share of Residual Purchasing - Category Buyers',
        'Index Against Expectation']})

rows = cross_purch_df.columns[:-1]

importance_against_expect = pl.from_numpy(stack, schema=rows).hstack(columns=cols)

In [91]:
(
    GT(importance_against_expect, rowname_col='Col')
    .tab_header(title='Importance Against Expectation',
                subtitle='Compare Actual Purchasing against Expectation Given General Purchasing Patterns')    
    .fmt_percent(rows=list(range(4)), decimals=0)
    .fmt_integer(rows=4)
    .data_color(
        rows=4,
        domain=[80, 170],
        palette=['white', 'rebeccapurple'],
    )
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('white'), 
        ],
        locations=[loc.body(0,[2, 3, 4])]
    )    
)

GT(_tbl_data=shape: (5, 6)
┌────────────┬───────────┬────────────┬────────────┬────────────┬─────────────────────────────────┐
│ Alpha      ┆ Bravo     ┆ Charlie    ┆ Delta      ┆ Other      ┆ Col                             │
│ ---        ┆ ---       ┆ ---        ┆ ---        ┆ ---        ┆ ---                             │
│ f64        ┆ f64       ┆ f64        ┆ f64        ┆ f64        ┆ str                             │
╞════════════╪═══════════╪════════════╪════════════╪════════════╪═════════════════════════════════╡
│ 0.688415   ┆ 0.184859  ┆ 0.080836   ┆ 0.037814   ┆ 0.008075   ┆ Share of Category Purchasing    │
│ 0.441003   ┆ 0.396457  ┆ 0.104456   ┆ 0.044311   ┆ 0.013773   ┆ Volume Market Share             │
│ 2.2094     ┆ 0.593287  ┆ 0.259434   ┆ 0.121362   ┆ 0.025917   ┆ Share of Residual Purchasing -… │
│ 0.788919   ┆ 0.709229  ┆ 0.186864   ┆ 0.079268   ┆ 0.024639   ┆ Share of Residual Purchasing -… │
│ 280.054233 ┆ 83.652453 ┆ 138.835568 ┆ 153.102396 ┆ 105.188478 ┆ Index Against Expectation       │
└────────────┴───────────┴────────────┴────────────┴────────────┴─────────────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x11b552210>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Col', type=<ColInfoTypeEnum.stub: 2>, column_label='Col', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b51c950>, _spanners=Spanners([]), _heading=Heading(title='Importance Against Expectation', subtitle='Compare Actual Purchasing against Expectation Given General Purchasing Patterns', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[4]), grpname=None, colname='Bravo', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f9f7fb')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[4]), grpname=None, colname='Charlie', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9b7abc')]), StyleInfo(locname=LocBody(columns='Delta', rows=[4]), grpname=None, colname='Delta', rownum=4, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8359ac')]), StyleInfo(locname=LocBody(columns='Other', rows=[4]), grpname=None, colname='Other', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d4c6e2')]), StyleInfo(locname=LocBody(columns='Col', rows=[4]), grpname=None, colname='Col', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_alig

In [92]:
index_ae = pl.DataFrame({'Index Against Expectation - By Volume Purchase': index_against_expect[1:],
                         'Brands': cross_purch_df.columns[1:-1]})

# Create the Altair chart
alt.Chart(index_ae).mark_bar().encode(
    x=alt.X("Index Against Expectation - By Volume Purchase:Q"),
    y=alt.Y("Brands:N"),
).properties(
    width=650,
    height=250,
    title={"text": 'Importance Against Expectation'},
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


Let us consider *Charlie*. With reference to Charlie's residual share of category purchasing not accounted for by Alpha, we see that it accounted for 26% of the category purchasing by Alpha buyers that did not go to Alpha. If the purchasing of Alpha buyers was consistent with overall market patterns (as reflected in the volume market shares), we would expect Charlie to account for 19% of their purchasing. We can therefore say that Charlie’s share of purchasing amongst the
buyers of Alpha is above expectation (when expectation is based on overall patterns of buying behaviour).

The **Index Against Expectation** essentially captures whether or not and to what extent volume purchases of Alpha's buyers went to its competitors above or below expectations / patterns observed in the overall market.

We see that, relative to market share, Bravo is less of threat to Alpha than we would expect (index = 84). Relative to market share, Charlie and Delta are purchased more by buyers of Alpha than we would expect.

##### Cross Purchasing - *By Spend*

We can repeat these analyses using spend rather than volume purchasing. 

We see, for example, that buyers of Alpha spent 71% of their category spend on Alpha. This is in contrast to the 69% of their category volume requirements satisfied by Alpha.

In [93]:
brand_spend_purchases = spend_pivot(grocery_lf, 1).fill_null(0)

# Step 1: Create ever buyers matrix
ever_buyers = brand_spend_purchases.select(
    [pl.when(pl.col(col) > 0).then(1).otherwise(0).alias(col) for col in brand_purchases.columns]
).to_numpy()

# Step 2: Compute cross purchase matrix
cross_purch_matrix = ever_buyers.T @ brand_spend_purchases
total_spend_purchases = np.sum(cross_purch_matrix, axis=1).reshape(5, 1)

# Step 3: Compute cross purchase matrix as a % of total spend of each brand
cross_purch = cross_purch_matrix / total_spend_purchases

# DataFrame conversion for tabular view
cross_purch_matrix_df = (
    pl.from_numpy(cross_purch_matrix, schema=brand_spend_purchases.columns)
    .with_columns(pl.Series('Brands', brand_spend_purchases.columns))
)

cross_purch_df = (
    pl.from_numpy(cross_purch, schema=brand_spend_purchases.columns)
    .with_columns(pl.Series('Brands', brand_spend_purchases.columns))
)

In [94]:
(
    GT(cross_purch_matrix_df, rowname_col='Brands')
    .tab_header(title='Cross Purchasing Analysis',
                subtitle='Total Category Spend by Buyers of the Brand (by row)')
    .tab_stubhead(label="Brands")
    .sub_missing(missing_text="")
    .fmt_integer()
    .data_color(
        domain=[0, 10_000],
        palette=['white', 'rebeccapurple'],
    ) 
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('lightgrey'),
            
        ],
        locations=[
            loc.body(0,0),
            loc.body(1,1),
            loc.body(2,2),
            loc.body(3,3),
            loc.body(4,4)
        ]
    )
    .tab_source_note(md("""**Read Row/Column as**: _Purchasers of [Brands] / Total Spend on [Brands]_"""))
)

GT(_tbl_data=shape: (5, 6)
┌──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────┐
│ Alpha        ┆ Bravo        ┆ Charlie     ┆ Delta       ┆ Other       ┆ Brands  │
│ ---          ┆ ---          ┆ ---         ┆ ---         ┆ ---         ┆ ---     │
│ f64          ┆ f64          ┆ f64         ┆ f64         ┆ f64         ┆ str     │
╞══════════════╪══════════════╪═════════════╪═════════════╪═════════════╪═════════╡
│ 33570.940452 ┆ 8913.120086  ┆ 2606.31     ┆ 1806.949983 ┆ 571.450025  ┆ Alpha   │
│ 9291.790116  ┆ 28603.350264 ┆ 2076.120019 ┆ 1008.139997 ┆ 1022.180028 ┆ Bravo   │
│ 4480.69004   ┆ 3853.970006  ┆ 5120.87003  ┆ 792.719992  ┆ 128.94      ┆ Charlie │
│ 2596.780043  ┆ 1148.630005  ┆ 806.349999  ┆ 3271.509972 ┆ 57.410001   ┆ Delta   │
│ 824.740008   ┆ 1337.130024  ┆ 118.380002  ┆ 74.030001   ┆ 1535.230041 ┆ Other   │
└──────────────┴──────────────┴─────────────┴─────────────┴─────────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x11b523ed0>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Brands', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b534bf0>, _spanners=Spanners([]), _heading=Heading(title='Cross Purchasing Analysis', subtitle='Total Category Spend by Buyers of the Brand (by row)', preheader=None), _stubhead='Brands', _source_notes=[Md(text='**Read Row/Column as**: _Purchasers of [Brands] / Total Spend on [Brands]_')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7141a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#baa4d1')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d7cae5')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f2eef7')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(col

In [95]:
(
    GT(cross_purch_df, rowname_col='Brands')
    .tab_header(title='Cross Purchasing Analysis',
                subtitle='% of Total Category Spend by Buyers of the Brand (by row)')
    .tab_stubhead(label="Brands")
    .sub_missing(missing_text="")
    .fmt_percent(decimals=0)
    .data_color(
        domain=[0, 0.8],
        palette=['white', 'rebeccapurple'],
    ) 
    .tab_source_note(md("""**Read Row/Column as**: _Purchasers of [Brands] / % Total Spend on [Brands]_"""))
)

GT(_tbl_data=shape: (5, 6)
┌──────────┬──────────┬──────────┬──────────┬──────────┬─────────┐
│ Alpha    ┆ Bravo    ┆ Charlie  ┆ Delta    ┆ Other    ┆ Brands  │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ str     │
╞══════════╪══════════╪══════════╪══════════╪══════════╪═════════╡
│ 0.707222 ┆ 0.187768 ┆ 0.054906 ┆ 0.038066 ┆ 0.012038 ┆ Alpha   │
│ 0.221225 ┆ 0.681007 ┆ 0.04943  ┆ 0.024002 ┆ 0.024337 ┆ Bravo   │
│ 0.311653 ┆ 0.268061 ┆ 0.35618  ┆ 0.055137 ┆ 0.008968 ┆ Charlie │
│ 0.329512 ┆ 0.145753 ┆ 0.10232  ┆ 0.41513  ┆ 0.007285 ┆ Delta   │
│ 0.212042 ┆ 0.343779 ┆ 0.030436 ┆ 0.019033 ┆ 0.39471  ┆ Other   │
└──────────┴──────────┴──────────┴──────────┴──────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x11b5519a0>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Brands', type=<ColInfoTypeEnum.stub: 2>, column_label='Brands', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b51fa70>, _spanners=Spanners([]), _heading=Heading(title='Cross Purchasing Analysis', subtitle='% of Total Category Spend by Buyers of the Brand (by row)', preheader=None), _stubhead='Brands', _source_notes=[Md(text='**Read Row/Column as**: _Purchasers of [Brands] / % Total Spend on [Brands]_')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#784ba5')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d5c7e3')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c3b0d7')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c0abd5')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d6c9e4')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#dbcfe7')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=

##### Importance Against Expectation - *By Spend*

We can create a spend-based importance against expectation plot, using value market share as the reference.

In [96]:
# Share of Category Purchasing - Alpha Brand Selected
share_of_cat_purch = cross_purch[0]

# Market Share - Volume
market_share = (
    brand_spend_purchases.sum() / # sum all columns -> returns a single row dataframe
    brand_spend_purchases.sum_horizontal().sum() # sum of total category volume sales -> retruns a scalar value
).to_numpy().reshape(5)

# Share of residual purchasing -- Alpha Buyers
# The percentage of the category purchasing not accounted for by Alpha that goes to each of the other brands
residual_purch_brand = share_of_cat_purch / (1 - share_of_cat_purch[0])

# Share of residual purchasing -- Category Buyers
# Residual share of category purchasing (across all category buyers) once Alpha is removed
# when we exclude Alpha, what percentage of (the remaining) category purchasing goes to each of the other brands)
residual_purch_category = market_share / (1 - market_share[0])

# Index against expectation
index_against_expect = 100 * residual_purch_brand/residual_purch_category

# Importance Against Expectations - Create DataFrame
stack = np.vstack((share_of_cat_purch, 
                   market_share, 
                   residual_purch_brand, 
                   residual_purch_category, 
                   index_against_expect))

cols = pl.DataFrame({'Col': ['Share of Category Purchasing', 
        'Volume Market Share', 
        'Share of Residual Purchasing - Alpha Buyers', 
        'Share of Residual Purchasing - Category Buyers',
        'Index Against Expectation']})

rows = cross_purch_df.columns[:-1]

importance_against_expect = pl.from_numpy(stack, schema=rows).hstack(columns=cols)

In [97]:
(
    GT(importance_against_expect, rowname_col='Col')
    .tab_header(title='Importance Against Expectation',
                subtitle='Compare Actual Purchasing against Expectation Given General Purchasing Patterns')    
    .fmt_percent(rows=list(range(4)), decimals=0)
    .fmt_integer(rows=4)
    .data_color(
        rows=4,
        domain=[80, 170],
        palette=['white', 'rebeccapurple'],
    )
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('white'), 
        ],
        locations=[loc.body(0,[2, 3, 4])]
    )    
)

GT(_tbl_data=shape: (5, 6)
┌────────────┬───────────┬────────────┬────────────┬────────────┬─────────────────────────────────┐
│ Alpha      ┆ Bravo     ┆ Charlie    ┆ Delta      ┆ Other      ┆ Col                             │
│ ---        ┆ ---       ┆ ---        ┆ ---        ┆ ---        ┆ ---                             │
│ f64        ┆ f64       ┆ f64        ┆ f64        ┆ f64        ┆ str                             │
╞════════════╪═══════════╪════════════╪════════════╪════════════╪═════════════════════════════════╡
│ 0.707222   ┆ 0.187768  ┆ 0.054906   ┆ 0.038066   ┆ 0.012038   ┆ Share of Category Purchasing    │
│ 0.465604   ┆ 0.396707  ┆ 0.071023   ┆ 0.045373   ┆ 0.021293   ┆ Volume Market Share             │
│ 2.415553   ┆ 0.641332  ┆ 0.187534   ┆ 0.130017   ┆ 0.041118   ┆ Share of Residual Purchasing -… │
│ 0.871272   ┆ 0.742347  ┆ 0.132903   ┆ 0.084906   ┆ 0.039844   ┆ Share of Residual Purchasing -… │
│ 277.244402 ┆ 86.392424 ┆ 141.105879 ┆ 153.130139 ┆ 103.197128 ┆ Index Against Expectation       │
└────────────┴───────────┴────────────┴────────────┴────────────┴─────────────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x11b522670>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Col', type=<ColInfoTypeEnum.stub: 2>, column_label='Col', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b51c410>, _spanners=Spanners([]), _heading=Heading(title='Importance Against Expectation', subtitle='Compare Actual Purchasing against Expectation Given General Purchasing Patterns', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[4]), grpname=None, colname='Bravo', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f4f1f8')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[4]), grpname=None, colname='Charlie', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9774ba')]), StyleInfo(locname=LocBody(columns='Delta', rows=[4]), grpname=None, colname='Delta', rownum=4, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8359ac')]), StyleInfo(locname=LocBody(columns='Other', rows=[4]), grpname=None, colname='Other', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d8cae5')]), StyleInfo(locname=LocBody(columns='Col', rows=[4]), grpname=None, colname='Col', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_alig

In [98]:
index_ae = pl.DataFrame({'Index Against Expectation - By Spend': index_against_expect[1:],
                         'Brands': cross_purch_df.columns[1:-1]})

# Create the Altair chart
alt.Chart(index_ae).mark_bar().encode(
    x=alt.X("Index Against Expectation - By Spend:Q"),
    y=alt.Y("Brands:N"),
).properties(
    width=650,
    height=250,
    title={"text": 'Importance Against Expectation'}
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Garbage Collect

In [99]:
exceptions = ['grocery_lf', 'sku_lf', 'kiwi_lf', 'In', 'exceptions', 'active_variables',
              'penetration', 'ppb', 'panel_size']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (pl.LazyFrame, pl.DataFrame, pl.Series, alt.Chart, alt.LayerChart, list, int, float, str, np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var

gc.collect()

531

### <span style="color:#956bbf">Exploring Dynamics in Buyer Behaviour</span>
---

Up to now, we have been characterizing buyer behavior in a given time period (i.e., one year), be it focusing on one brand or multiple brands. We now consider some standard analyses that give insight into the dynamics of buyer behavior from period to period. We first consider the case of established products and then turn our attention to the analysis of new product buying behavior.

#### Established Products

We are interested in summarizing how buyer behavior varies across consecutive periods. We first consider how **temporal variations in total sales** can be understood by **decomposing total sales**. Next we explore **temporal variations in customer-level purchasing** by examining how the distribution of purchasing in one period varies as a function of the level of purchasing in the previous period. Finally, we consider a summary measure of period-to-period purchasing called the **repeat rate**.

##### Understanding Temporal Variations in Sales

Most firms have systems that will report sales over time. As we try to make sense of any observed changes, it is helpful to note a **fundamental (multiplicative) sales decomposition**. For any time period,

$$
  \begin{aligned}
    \text{Sales}   &= \text{\# households (HHs) in the country} \\
                   &= \times \text{proportion of HHs buying the brand (penetration)} \\
                   &= \times \text{\# purchase occasions per buyer (PPB)} \\
                   &= \times \text{\# packs per purchase} \\
                   &= \times \text{weight or price per pack} \\
  \end{aligned}
$$

There is nothing magical about this specific decomposition. We can create variations on a theme that are more relevant for the specific analysis setting at hand. For example, suppose we are doing an analysis at the brand level, where the SKUs associated with the brand come in different sizes. Furthermore, suppose the time period is sufficiently small that households make only one purchase per period, if at all. (In other words, PPB = 1). A more relevant decomposition would be

$$
  \begin{aligned}
    \text{Sales (\$)}   &= \text{\# households (HHs) in the country} \\
                   &= \times \text{proportion of HHs buying the brand (penetration)} \\
                   &= \times \text{average volume per purchase} \\
                   &= \times \text{average price per unit of volume} \\
  \end{aligned}
$$

The product of the last two quantities is often called **average order value (AOV)**.

$$AOV = \text{average volume per purchase} \times \text{average price per unit of volume}$$

Recall the plot of Alpha's revenue we created earlier:

In [100]:
weekly_plot(dataframe=weekly_spend_summary('Alpha', grocery_lf).collect(), 
            y='Weekly Spend', 
            title='Plot of weekly revenue for Alpha', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


We observe some weeks where this is a massive increase in revenue. How much of this is due to an increase in penetration versus, say, buyers simply buying more product on a given purchase occasion? Let us explore this using the second decomposition given above.

1) We start by computing Alpha’s **weekly penetration numbers**:
   - Check whether purchase purchase occasions per buyer (PPB) = 1.
   - We do this by grouping the panel data by '`week`' and aggregating to get the number of unique '`trans_id`' and '`panel_id`' per week. If the '`num_trans`' and '`num_buyers`' is the same for each week, then no panellist made more than one purchase in any given week, which means PPB = 1.
   - We produce a new dataframe that is grouped by 104 weeks and aggregates the count of unique number of panellists (*weekly penetration*) and number of transaction IDs each week.
   - Output: The number of panellists that made at least one purchase of Alpha; the total number of category purchase occasions on which Alpha was purchased, for each week.
2) The next step is to add the **weekly volume and revenue** numbers: 
   - Join SKU weights data with panel data, group panel data by `week`, compute `volume` and `spend`.
   - Output: Alpha’s (dollar and volume) sales.
3) Next we compute the numbers associated with the **revenue decomposition**:
   - **Weekly penetration** is simply the number of panellists active in any given week divided by the size of the panel.
   - **Average order value** is simply total revenue for any given week divided by the number of panellists that made at least one purchase of Alpha in that week.
     - As noted above, the AOV quantity can be decomposed into *average order volume* and *average price per unit volume* (in this case, kg).
   - **Average order volume** is simply (total) volume sold in any given week divided by the number of panellists that made at least one purchase of Alpha in that week.
   - **Average price per unit volume** is simply total revenue for any given week divided by (total) volume sold in that week.

In [101]:
(
    grocery_lf
    .filter(pl.col('brand') == 'Alpha')
    .group_by("week")
    .agg(
        pl.col("trans_id").n_unique().alias("num_trans"),
        pl.col("panel_id").n_unique().alias("num_buyers")
    ).with_columns(
        (pl.col('num_trans') / pl.col('num_buyers')).alias('ppb')
    )
    .sort('week')
    .filter(pl.col('ppb') == 1)
    
).collect()

week,num_trans,num_buyers,ppb
u16,u32,u32,f64
1,93,93,1.0
2,166,166,1.0
3,143,143,1.0
4,304,304,1.0
5,107,107,1.0
…,…,…,…
100,206,206,1.0
101,75,75,1.0
102,105,105,1.0


We note that the number of transactions (`num_trans`) associated with each week equals the number of panellists (`num_buyers`) associated with each week. In other words, no panellist made more than one purchase in any given week, which means PPB = 1 (`num_trans` / `num_buyers`), as assumed above.

In [102]:
# Filter for rows where the brand is "Alpha"
alpha_lf = grocery_lf.filter(pl.col("brand") == "Alpha")

# Aggregating to get the number of unique trans_id and panel_id per week
alpha_weekly_trans = (
    alpha_lf
    .group_by("week")
    .agg(
        pl.col("trans_id").n_unique().alias("num_trans"),
        pl.col("panel_id").n_unique().alias("num_buyers")
    )    
)

# Aggregating to get the sum of spend and volume per week
alpha_weekly_spend_vol = (
    alpha_lf
    .with_columns(((pl.col('units') * pl.col('price'))).alias('spend'))
    .join(other=sku_lf, on='sku_id')
    .with_columns((((pl.col('units') * pl.col('weight'))/1000)).alias('volume'))
    .group_by('week').agg(
        pl.col("spend").sum().alias("spend"),
        pl.col("volume").sum().alias("volume")        
    )
)

# The alpha_weekly_trans & alpha_weekly_spend_vol are joined.
# alpha_weekly_summary now contains week, num_trans, num_buyers, spend, and volume
alpha_weekly_summary = alpha_weekly_trans.join(alpha_weekly_spend_vol, on="week").sort('week')

alpha_weekly = (
    alpha_weekly_summary
    .with_columns(
        # Weekly penetration = num_buyer / panel_sizes
        (pl.col('num_buyers') / panel_size).alias('penet'),
        # PPB = num_trans / num_buyer
        (pl.col('num_trans') / pl.col('num_buyers')).alias('ppb'),
        # Average Order Value = spend / num_trans
        (pl.col('spend') / pl.col('num_trans')).alias('aov'),
        # Average Order Value = spend / num_trans
        (pl.col('volume') / pl.col('num_trans')).alias('aovol'),
        # Average price per unit volume = spend / volume
        (pl.col('spend') / pl.col('volume')).alias('avg_price_kg')
    ).select(
        'week', 'spend', 'penet', 'aov', 'aovol', 'avg_price_kg'
        
    )
)

Now we compute the correlations between weekly revenue and the components of its (multiplicative) decomposition across the two years.

**Correlations between Weekly Revenue & its Components - Over 2 Years**

In [103]:
corr_matrix = alpha_weekly.select(pl.all().exclude('week')).collect().corr()

relabel = pl.Series(['Revenue', 'Penetration', 'AOV', 'AOVOL', 'Price/Unit'])

corr_matrix = corr_matrix.with_columns(pl.Series(relabel).alias("Decomposition"))

(
    GT(corr_matrix, rowname_col='Decomposition')
    .tab_header(title='Correlation Matrix of Revenue and the Components of its Decomposition',
                subtitle='Correlations between weekly revenue and the components of its (multiplicative) decomposition')
    .cols_label(
        spend=relabel[0],
        penet=relabel[1],
        aov=relabel[2],
        aovol=relabel[3],
        avg_price_kg=relabel[4]
    )
    .cols_align(align='center')
    .fmt_number(decimals=3)
    .data_color(
        domain=[-1, 1],
        palette=['orange', 'white', 'rebeccapurple'],
    ) 
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('lightgrey'),
            
        ],
        locations=[
            loc.body(0,0),
            loc.body(1,1),
            loc.body(2,2),
            loc.body(3,3),
            loc.body(4,4)
        ]
    )
)

GT(_tbl_data=shape: (5, 6)
┌──────────┬───────────┬───────────┬───────────┬──────────────┬───────────────┐
│ spend    ┆ penet     ┆ aov       ┆ aovol     ┆ avg_price_kg ┆ Decomposition │
│ ---      ┆ ---       ┆ ---       ┆ ---       ┆ ---          ┆ ---           │
│ f64      ┆ f64       ┆ f64       ┆ f64       ┆ f64          ┆ str           │
╞══════════╪═══════════╪═══════════╪═══════════╪══════════════╪═══════════════╡
│ 1.0      ┆ 0.981849  ┆ 0.542343  ┆ 0.758487  ┆ -0.58958     ┆ Revenue       │
│ 0.981849 ┆ 1.0       ┆ 0.386869  ┆ 0.70345   ┆ -0.654035    ┆ Penetration   │
│ 0.542343 ┆ 0.386869  ┆ 1.0       ┆ 0.691476  ┆ -0.073048    ┆ AOV           │
│ 0.758487 ┆ 0.70345   ┆ 0.691476  ┆ 1.0       ┆ -0.754159    ┆ AOVOL         │
│ -0.58958 ┆ -0.654035 ┆ -0.073048 ┆ -0.754159 ┆ 1.0          ┆ Price/Unit    │
└──────────┴───────────┴───────────┴───────────┴──────────────┴───────────────┘, _body=<great_tables._gt_data.Body object at 0x11b56b2a0>, _boxhead=Boxhead([ColInfo(var='spend', type=<ColInfoTypeEnum.default: 1>, column_label='Revenue', column_align='center', column_width=None), ColInfo(var='penet', type=<ColInfoTypeEnum.default: 1>, column_label='Penetration', column_align='center', column_width=None), ColInfo(var='aov', type=<ColInfoTypeEnum.default: 1>, column_label='AOV', column_align='center', column_width=None), ColInfo(var='aovol', type=<ColInfoTypeEnum.default: 1>, column_label='AOVOL', column_align='center', column_width=None), ColInfo(var='avg_price_kg', type=<ColInfoTypeEnum.default: 1>, column_label='Price/Unit', column_align='center', column_width=None), ColInfo(var='Decomposition', type=<ColInfoTypeEnum.stub: 2>, column_label='Decomposition', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b51f410>, _spanners=Spanners([]), _heading=Heading(title='Correlation Matrix of Revenue and the Components of its Decomposition', subtitle='Correlations between weekly revenue and the components of its (multiplicative) decomposition', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='spend', rows=[0]), grpname=None, colname='spend', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='spend', rows=[1]), grpname=None, colname='spend', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69379b')]), StyleInfo(locname=LocBody(columns='spend', rows=[2]), grpname=None, colname='spend', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ac90c8')]), StyleInfo(locname=LocBody(columns='spend', rows=[3]), grpname=None, colname='spend', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8b64b2')]), StyleInfo(locname=LocBody(columns='spend', rows=[4]), grpname=None, colname='spend', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffca69')]), StyleInfo(locname=LocBody(columns='penet', rows=[0]), grpname=None, colname='penet', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69379b')]),

Below is a plot of the penetration numbers. Comparing this with Alpha's weekly revenue plot over the two years, we see that the fluctuations in revenue go hand-in-hand with fluctuations in penetration. To make this comparison clearer, we overlay the two series of numbers as well. The correlation between these to quantities is 0.98. It would appear that the key driver of revenue increases is simply more people buying the brand that week.

In [104]:
weekly_plot(dataframe=alpha_weekly.select('week', 'penet').collect(), 
            y='penet', 
            title='Alpha - Weekly Penetration', 
            y_axis_label='Penetration',
            pct=True,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [105]:
source = alpha_weekly.select('week', 'spend', 'penet').collect()

base = alt.Chart(source).mark_line().transform_fold(
    ['Penetration', 'Revenue'],
    as_=['Legend', 'Value']
).encode(
    alt.Color('Legend:N'),
    alt.X('week', axis=alt.Axis(
        values=np.arange(0, 104 + 1, 13), # Explicitly specify quarter-end weeks
        labelExpr="datum.value", # Show only these labels
        title='Week'))
)

spend_line = base.transform_filter(
    alt.datum.Legend == 'Revenue'
    ).encode(
        y = alt.Y(
            'spend:Q', # Q = a continuous real-valued quantity
            axis=alt.Axis(format="$,.0f")).title('Revenue ($)')
)

penet_line = base.transform_filter(
    alt.datum.Legend == 'Penetration'
).encode(
    y = alt.Y(
        'penet:Q', # Q = a continuous real-valued quantity
        axis=alt.Axis(format=",.0%")).title('Penetration')
)

alt.layer(spend_line, penet_line).resolve_scale(
    y='independent'
).properties(
    width=600,
    height=250,
    title='Weekly Revenue & Penetration for Alpha'    
).configure_view(stroke=None).configure_axisY(grid=True)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


However, the lack of a perfect correlation means there is some variability in average order value that is not highly correlated with penetration. We plot this quantity below. (The correlation between penetration and average order value is 0.39.) In order to get a sense of what lies behind the variability in average order value, we also plot weekly average order volume and average price per kg, respectively.

In [106]:
def weekly_plot(dataframe, y, year=2, title="", y_axis_label="", fmt='currency'):
    
    if fmt == 'currency':
        format = "$,.1f"
    elif fmt == 'percent':
        format = ",.0%"
    else:
        format = ",.1f"
    
    chart = alt.Chart(dataframe).mark_line(strokeWidth=1).encode(
        x = alt.X(
            'week',
            axis=alt.Axis(
                values=np.arange(0, (year*52) + 1, 13), # Explicitly specify quarter-end weeks
                labelExpr="datum.value", # Show only these labels
                title='Week'
            )
        ),
        y = alt.Y(
            f'{y}:Q', # Q = a continuous real-valued quantity
            title=y_axis_label,
            axis=alt.Axis(format=format)
        )
    ).properties(
        width=650,
        height=250,
        title=title
    ).configure_view(
        stroke=None
    )

    return chart 

In [107]:
weekly_plot(dataframe=alpha_weekly.select('week', 'aov').collect(), 
            y='aov', 
            title='Alpha - Weekly Average Order Value', 
            y_axis_label='Average Order Value ($)',
            fmt='currency')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [108]:
weekly_plot(dataframe=alpha_weekly.select('week', 'aovol').collect(), 
            y='aovol', 
            title='Alpha - Weekly Average Order Volume (kg)', 
            y_axis_label='Average Order Volume (kg)',
            fmt='number')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [109]:
weekly_plot(dataframe=alpha_weekly.select('week', 'avg_price_kg').collect(), 
            y='avg_price_kg', 
            title='Alpha - Weekly Average Price/kg', 
            y_axis_label='Average $/kg ',
            fmt='currency')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


Looking at the weekly average price per kg plot, there was much less variability in price/kg in weeks 1–52 compared to weeks 53–104. It would appear that there was some change in promotion policy between years 1 and 2. We do not have the data to explore this further, however. 

**Correlations between Weekly Revenue & its Components - Year 1**

In [110]:
corr_matrix = alpha_weekly.filter(pl.col('week') <= 52).select(pl.all().exclude('week')).collect().corr()

relabel = pl.Series(['Revenue', 'Penetration', 'AOV', 'AOVOL', 'Price/Unit'])

corr_matrix = corr_matrix.with_columns(pl.Series(relabel).alias("Decomposition"))

(
    GT(corr_matrix, rowname_col='Decomposition')
    .tab_header(title='Correlation Matrix of Revenue and the Components of its Decomposition',
                subtitle='Correlations between weekly revenue and the components of its (multiplicative) decomposition')
    .cols_label(
        spend=relabel[0],
        penet=relabel[1],
        aov=relabel[2],
        aovol=relabel[3],
        avg_price_kg=relabel[4]
    )
    .cols_align(align='center')
    .fmt_number(decimals=3)
    .data_color(
        domain=[-1, 1],
        palette=['orange', 'white', 'rebeccapurple'],
    ) 
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('lightgrey'),
            
        ],
        locations=[
            loc.body(0,0),
            loc.body(1,1),
            loc.body(2,2),
            loc.body(3,3),
            loc.body(4,4)
        ]
    )
)

GT(_tbl_data=shape: (5, 6)
┌───────────┬───────────┬───────────┬───────────┬──────────────┬───────────────┐
│ spend     ┆ penet     ┆ aov       ┆ aovol     ┆ avg_price_kg ┆ Decomposition │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---          ┆ ---           │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64          ┆ str           │
╞═══════════╪═══════════╪═══════════╪═══════════╪══════════════╪═══════════════╡
│ 1.0       ┆ 0.990195  ┆ 0.787646  ┆ 0.839754  ┆ -0.533591    ┆ Revenue       │
│ 0.990195  ┆ 1.0       ┆ 0.707296  ┆ 0.777812  ┆ -0.553106    ┆ Penetration   │
│ 0.787646  ┆ 0.707296  ┆ 1.0       ┆ 0.956489  ┆ -0.349201    ┆ AOV           │
│ 0.839754  ┆ 0.777812  ┆ 0.956489  ┆ 1.0       ┆ -0.606428    ┆ AOVOL         │
│ -0.533591 ┆ -0.553106 ┆ -0.349201 ┆ -0.606428 ┆ 1.0          ┆ Price/Unit    │
└───────────┴───────────┴───────────┴───────────┴──────────────┴───────────────┘, _body=<great_tables._gt_data.Body object at 0x11b53e440>, _boxhead=Boxhead([ColInfo(var='spend', type=<ColInfoTypeEnum.default: 1>, column_label='Revenue', column_align='center', column_width=None), ColInfo(var='penet', type=<ColInfoTypeEnum.default: 1>, column_label='Penetration', column_align='center', column_width=None), ColInfo(var='aov', type=<ColInfoTypeEnum.default: 1>, column_label='AOV', column_align='center', column_width=None), ColInfo(var='aovol', type=<ColInfoTypeEnum.default: 1>, column_label='AOVOL', column_align='center', column_width=None), ColInfo(var='avg_price_kg', type=<ColInfoTypeEnum.default: 1>, column_label='Price/Unit', column_align='center', column_width=None), ColInfo(var='Decomposition', type=<ColInfoTypeEnum.stub: 2>, column_label='Decomposition', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b5c1d90>, _spanners=Spanners([]), _heading=Heading(title='Correlation Matrix of Revenue and the Components of its Decomposition', subtitle='Correlations between weekly revenue and the components of its (multiplicative) decomposition', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='spend', rows=[0]), grpname=None, colname='spend', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='spend', rows=[1]), grpname=None, colname='spend', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#68359a')]), StyleInfo(locname=LocBody(columns='spend', rows=[2]), grpname=None, colname='spend', rownum=2, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865eaf')]), StyleInfo(locname=LocBody(columns='spend', rows=[3]), grpname=None, colname='spend', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7f54a9')]), StyleInfo(locname=LocBody(columns='spend', rows=[4]), grpname=None, colname='spend', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffcf77')]), StyleInfo(locname=LocBody(columns='penet', rows=[0]), grpname=None, colname='penet', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#

**Correlations between Weekly Revenue & its Components - Year 2**

In [111]:
corr_matrix = alpha_weekly.filter(pl.col('week') >= 53).select(pl.all().exclude('week')).collect().corr()

relabel = pl.Series(['Revenue', 'Penetration', 'AOV', 'AOVOL', 'Price/Unit'])

corr_matrix = corr_matrix.with_columns(pl.Series(relabel).alias("Decomposition"))

(
    GT(corr_matrix, rowname_col='Decomposition')
    .tab_header(title='Correlation Matrix of Revenue and the Components of its Decomposition',
                subtitle='Correlations between weekly revenue and the components of its (multiplicative) decomposition')
    .cols_label(
        spend=relabel[0],
        penet=relabel[1],
        aov=relabel[2],
        aovol=relabel[3],
        avg_price_kg=relabel[4]
    )
    .cols_align(align='center')
    .fmt_number(decimals=3)
    .data_color(
        domain=[-1, 1],
        palette=['orange', 'white', 'rebeccapurple'],
    ) 
    .tab_style(
        style=[
            style.text(color="white"),
            style.fill('lightgrey'),
            
        ],
        locations=[
            loc.body(0,0),
            loc.body(1,1),
            loc.body(2,2),
            loc.body(3,3),
            loc.body(4,4)
        ]
    )
)

GT(_tbl_data=shape: (5, 6)
┌───────────┬───────────┬───────────┬───────────┬──────────────┬───────────────┐
│ spend     ┆ penet     ┆ aov       ┆ aovol     ┆ avg_price_kg ┆ Decomposition │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---          ┆ ---           │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64          ┆ str           │
╞═══════════╪═══════════╪═══════════╪═══════════╪══════════════╪═══════════════╡
│ 1.0       ┆ 0.977958  ┆ 0.413588  ┆ 0.76995   ┆ -0.685022    ┆ Revenue       │
│ 0.977958  ┆ 1.0       ┆ 0.227083  ┆ 0.710412  ┆ -0.748287    ┆ Penetration   │
│ 0.413588  ┆ 0.227083  ┆ 1.0       ┆ 0.602068  ┆ -0.034845    ┆ AOV           │
│ 0.76995   ┆ 0.710412  ┆ 0.602068  ┆ 1.0       ┆ -0.804148    ┆ AOVOL         │
│ -0.685022 ┆ -0.748287 ┆ -0.034845 ┆ -0.804148 ┆ 1.0          ┆ Price/Unit    │
└───────────┴───────────┴───────────┴───────────┴──────────────┴───────────────┘, _body=<great_tables._gt_data.Body object at 0x11b53f070>, _boxhead=Boxhead([ColInfo(var='spend', type=<ColInfoTypeEnum.default: 1>, column_label='Revenue', column_align='center', column_width=None), ColInfo(var='penet', type=<ColInfoTypeEnum.default: 1>, column_label='Penetration', column_align='center', column_width=None), ColInfo(var='aov', type=<ColInfoTypeEnum.default: 1>, column_label='AOV', column_align='center', column_width=None), ColInfo(var='aovol', type=<ColInfoTypeEnum.default: 1>, column_label='AOVOL', column_align='center', column_width=None), ColInfo(var='avg_price_kg', type=<ColInfoTypeEnum.default: 1>, column_label='Price/Unit', column_align='center', column_width=None), ColInfo(var='Decomposition', type=<ColInfoTypeEnum.stub: 2>, column_label='Decomposition', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b5c0fb0>, _spanners=Spanners([]), _heading=Heading(title='Correlation Matrix of Revenue and the Components of its Decomposition', subtitle='Correlations between weekly revenue and the components of its (multiplicative) decomposition', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='spend', rows=[0]), grpname=None, colname='spend', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='spend', rows=[1]), grpname=None, colname='spend', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69379b')]), StyleInfo(locname=LocBody(columns='spend', rows=[2]), grpname=None, colname='spend', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c0abd5')]), StyleInfo(locname=LocBody(columns='spend', rows=[3]), grpname=None, colname='spend', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8962b0')]), StyleInfo(locname=LocBody(columns='spend', rows=[4]), grpname=None, colname='spend', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffc150')]), StyleInfo(locname=LocBody(columns='penet', rows=[0]), grpname=None, colname='penet', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#

**Annual Revenue Decomposition**

This same logic can be used to analyse change across longer time periods. For example, we note from our analysis earlier that Alpha’s revenue increased from \$33,571 in year 1 to \$35,251 in year 2, a 5% increase. As we are looking at a **static sample** (i.e., the same group of panellists over the two years), the **increase cannot be due to population growth**. Are we observing this growth because more households are buying the product (increase in buyers), and/or because those that are buying it are buying it more often (increase in frequency), and/or because the average spend per transaction has increased (increase in AOV)?

In order to explore this, we will use the following (multiplicative) sales decomposition:
$$
  \begin{aligned}
    \text{Annual Revenue}   &= \text{\# households (HHs) in the panel} \\
                   &= \times \text{proportion of HHs buying the brand (penetration)} \\
                   &= \times \text{\# purchase occasions per buyer (PPB)} \\
                   &= \times \text{average order volume} \\
                   &= \times \text{average price per unit of volume} \\
  \end{aligned}
$$

In order to perform a similar decomposition of annual revenue, we first need to create a dataset that summarises, for each year, (1) the number of panellists that made at least one purchase of Alpha, (2) the total number of category purchase occasions on which Alpha was purchased, and (3) Alpha’s (dollar and volume) sales. We use the same logic as above, aggregating by year as opposed to week.

1) compute is the number of panellists that made at least one transaction
2) compute total number of category purchase occasions (number of transactions) on which Alpha was purchased 
3) compute total revenue and volume for Alpha by year
4) compute the components of the revenue decomposition
   - **Penetration** is the number of households that made at least purchase of Alpha in the year divided by the size of the panel.
   - **PPB** is the total number of transactions associated with Alpha divided by the number of households that made at least purchase of Alpha in the year.
   - **Average Order Value** is annual revenue divided by the total number of transactions associated with Alpha.
   - **Average Order Volume** is annual total volume divided by the total number of transactions associated with Alpha.
   - **Average price/kg** is revenue divided by total volume. 
5) Computing the percentage changes

In [112]:
alpha_yearly_summary = (
    grocery_lf
    .filter(pl.col('brand') == 'Alpha')
    .join(other=sku_lf, on='sku_id')
    .with_columns(
        (pl.col('units') * pl.col('price')).cast(pl.Float64).alias('spend'),
        (pl.col('units') * pl.col('weight')/1000).alias('volume'),
        (pl.col('week') / 52).ceil().cast(pl.UInt16).alias('year')
    ).group_by('year')
    .agg(
        pl.col('trans_id').n_unique().alias('num_trans'),
        pl.col('panel_id').n_unique().alias('num_buyers'),
        pl.col('spend').sum(),
        pl.col('volume').sum()
    )
)

alpha_yearly = (
    alpha_yearly_summary
    .with_columns(
        # Yearly penetration = num_buyer / panel_sizes
        (pl.col('num_buyers') / panel_size).alias('penet'),
        # PPB = num_trans / num_buyer
        (pl.col('num_trans') / pl.col('num_buyers')).alias('ppb'),
        # Average Order Value = spend / num_trans
        (pl.col('spend') / pl.col('num_trans')).alias('aov'),
        # Average Order Value = spend / num_trans
        (pl.col('volume') / pl.col('num_trans')).alias('aovol'),
        # Average price per unit volume = spend / volume
        (pl.col('spend') / pl.col('volume')).alias('avg_price_kg')
    ).with_columns(pl.col('year').cast(pl.String))
    .collect()
)

alpha_yearly_T = alpha_yearly.transpose(
    include_header=True, header_name='Summary', column_names='year'
)

alpha_yearly_pct_change = (
    alpha_yearly
    .with_columns(pl.col('*').exclude('year').pct_change())
    .transpose(include_header=True, header_name='Summary', column_names='year')  
    .select('Summary' , pl.col('2').alias('pct_change'))  
)

alpha_yearly_T = alpha_yearly_T.join(alpha_yearly_pct_change, on='Summary')

In [113]:
relabel = pl.Series(['# of Trans.', 
                     '# of Buyers',
                     'Revenue',
                     'Volume',
                     'Penetration', 
                     'PPB',
                     'AOV', 
                     'AOVOL', 
                     'Avg. Price/Unit'])

group = pl.Series(['Sales Summary']*4 + ['Revenue Decomposition']*5)

alpha_yearly_T = alpha_yearly_T.with_columns(
    pl.Series(relabel).alias('Summary'), 
    pl.Series(group).alias('Group')
)

(
    GT(alpha_yearly_T, rowname_col='Summary', groupname_col='Group')
    .tab_header(title='Alpha - Annual Summary',
                subtitle='Transactions, Buyers, Volume, Revenue & Revenue Decomposition')
    .cols_label({'1': 'Year 1', 
                 '2': 'Year 2',
                 'pct_change': '% Change'})
    .fmt_integer(rows=[0,1])
    .fmt_number(rows=[2,3,5,7])
    .fmt_currency(rows=[6,8])
    .fmt_percent(rows=4)
    .fmt_percent(columns=[3])
    .data_color(
        columns=3,
        domain=[-1,1],
        palette=['orange', 'white', 'rebeccapurple']
    )
)

GT(_tbl_data=shape: (9, 5)
┌─────────────────┬──────────────┬─────────────┬────────────┬───────────────────────┐
│ Summary         ┆ 1            ┆ 2           ┆ pct_change ┆ Group                 │
│ ---             ┆ ---          ┆ ---         ┆ ---        ┆ ---                   │
│ str             ┆ f64          ┆ f64         ┆ f64        ┆ str                   │
╞═════════════════╪══════════════╪═════════════╪════════════╪═══════════════════════╡
│ # of Trans.     ┆ 9060.0       ┆ 9240.0      ┆ 0.019868   ┆ Sales Summary         │
│ # of Buyers     ┆ 2624.0       ┆ 2759.0      ┆ 0.051448   ┆ Sales Summary         │
│ Revenue         ┆ 33570.940269 ┆ 35250.75023 ┆ 0.050038   ┆ Sales Summary         │
│ Volume          ┆ 9166.25      ┆ 10346.4     ┆ 0.128749   ┆ Sales Summary         │
│ Penetration     ┆ 0.522605     ┆ 0.549492    ┆ 0.051448   ┆ Revenue Decomposition │
│ PPB             ┆ 3.452744     ┆ 3.34904     ┆ -0.030035  ┆ Revenue Decomposition │
│ AOV             ┆ 3.705402     ┆ 3.815016    ┆ 0.029582   ┆ Revenue Decomposition │
│ AOVOL           ┆ 1.011727     ┆ 1.11974     ┆ 0.106761   ┆ Revenue Decomposition │
│ Avg. Price/Unit ┆ 3.662451     ┆ 3.407055    ┆ -0.069734  ┆ Revenue Decomposition │
└─────────────────┴──────────────┴─────────────┴────────────┴───────────────────────┘, _body=<great_tables._gt_data.Body object at 0x11b53e120>, _boxhead=Boxhead([ColInfo(var='Summary', type=<ColInfoTypeEnum.stub: 2>, column_label='Summary', column_align='left', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='Year 1', column_align='right', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='Year 2', column_align='right', column_width=None), ColInfo(var='pct_change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None), ColInfo(var='Group', type=<ColInfoTypeEnum.row_group: 3>, column_label='Group', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x11b5ea210>, _spanners=Spanners([]), _heading=Heading(title='Alpha - Annual Summary', subtitle='Transactions, Buyers, Volume, Revenue & Revenue Decomposition', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='pct_change', rows=[0]), grpname=None, colname='pct_change', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fcfbfd')]), StyleInfo(locname=LocBody(columns='pct_change', rows=[1]), grpname=None, colname='pct_change', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f7f5fa')]), StyleInfo(locname=LocBody(columns='pct_change', rows=[2]), grpname=None, colname='pct_change', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f7f5fa')]), StyleInfo(locname=LocBody(columns='pct_change', rows=[3]), grpname=None, colname='pct_change', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ebe5f2')]), StyleInfo(locname=LocBody(columns='pct_change', rows=[4]), grpname=None, colname='pct_change', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f7f5fa')]), StyleInfo(locname=LocBody(columns='pct_change', rows=[5]), grpname=None, colname='pct_

We note that the *5% increase in revenue* from year 1 to year 2 is associated with a *5% increase in the number of households making at least one purchase* of Alpha in the year. While PPB drops, average order value increases, with these two changes effectively cancelling out each other. (The product of these two quantities changes by one cent between the two years. i.e, $(PPB_{2} \times AOV_{2})-(PPB_{1} \times AOV_{1})$) While the average price/kg drops from year 1 to year 2, this is more than compensated by the increase in average order volume, resulting in a 3% increase in average order value between the two years (AOR = Average Order Volume x Avearge Price/Unit).

##### Temporal Variation in Customer-level Purchasing

We have observed that the number of buyers has increased, yet the average number of transactions (PPB) has dropped. Does this mean the “new” buyers are light buyers? Or are the existing buyers buying less?

In order to dig deeper, we need to examine **temporal variation in customer-level purchasing**. A natural starting point is to examine the joint distribution of purchasing for two consecutive periods.

1) We first need to create a dataset that documents the number of times Alpha was purchased in years 1 and 2 by each panellist.

In [315]:
alpha_yearly_trans = (
    grocery_lf
    .filter(pl.col('brand') == 'Alpha')
    .with_columns((pl.col('week') / 52).ceil().cast(pl.UInt16).alias('year'))
    # Aggregate to count unique 'trans_id' by 'panel_id' and 'year'
    .group_by('panel_id','year')
    .agg(pl.col('trans_id').n_unique().alias('num_trans'))
)

alpha_yearly_trans = (
    # Reshape from long format to wide format
    # Pivoting the dataframe based on 'year' to create a wide format.
    alpha_yearly_trans.collect().pivot(
        on='year',          # Each unique year will create a new column
        values='num_trans', # The column to aggregate
        index='panel_id'    # Rows will be indexed by 'panel_id'
    )
    # Rename the columns to 'Year 1' and 'Year 2'
    .rename({'1': 'Year 1', '2': 'Year 2'})   
    # Replace any null values with 0
    .fill_null(0) 
)

# create a basic joint distribution
joint_dist_trans = (
    alpha_yearly_trans
    .group_by('Year 1', 'Year 2')
    .agg(pl.col('panel_id').count().alias('count'))
)

# add in the number of panellists that made no purchase of Alpha in either year
tmp = panel_size - joint_dist_trans.select((pl.col('count')).sum()).item(0,0)
zero_purch = pl.DataFrame({'Year 1': 0, 'Year 2': 0, 'count': tmp})

# We right censor the distribution at 10+
joint_dist_trans = (
    joint_dist_trans.with_columns(
        pl.when(pl.col('Year 1') > 9)
        .then(10)
        .otherwise(pl.col('Year 1'))
        .alias('Year 1')
    )
    .with_columns(
        pl.when(pl.col('Year 2') > 9)
        .then(10)    
        .otherwise(pl.col('Year 2'))
        .alias('Year 2')   
    )
    .vstack(zero_purch.cast(pl.UInt32))
    .sort('Year 2')
    .pivot(
        index='Year 1',
        on='Year 2',
        values='count',
        aggregate_function='sum'  
    )
    .rename({'Year 1': 'Year'}) 
    .sort('Year')   
    .fill_null(0)    
)

In [318]:
relabel = pl.Series([f'{i}' if i < 10 else '10+' for i in range(11)])
joint_dist_trans = joint_dist_trans.select(pl.col('*').exclude('Year')).with_columns(relabel.alias('Year'))
(
    GT(joint_dist_trans, rowname_col='Year')
    .tab_header(title='Joint Distribution of the Purchasing of Alpha in Year 1 & 2')
    .tab_stubhead(label='Year')
    .cols_label({'10': '10+'})
    .cols_align(align='center')
    .fmt_integer()
    .data_color(
        domain=[0, 1900],
        palette=['white', 'rebeccapurple'],
    ) 
)

GT(_tbl_data=shape: (11, 12)
┌──────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬──────┐
│ 0    ┆ 1   ┆ 2   ┆ 3   ┆ … ┆ 8   ┆ 9   ┆ 10  ┆ Year │
│ ---  ┆ --- ┆ --- ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ ---  │
│ u32  ┆ u32 ┆ u32 ┆ u32 ┆   ┆ u32 ┆ u32 ┆ u32 ┆ str  │
╞══════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪══════╡
│ 1879 ┆ 342 ┆ 105 ┆ 39  ┆ … ┆ 0   ┆ 0   ┆ 0   ┆ 0    │
│ 259  ┆ 201 ┆ 128 ┆ 79  ┆ … ┆ 2   ┆ 0   ┆ 1   ┆ 1    │
│ 83   ┆ 120 ┆ 108 ┆ 80  ┆ … ┆ 3   ┆ 1   ┆ 0   ┆ 2    │
│ 25   ┆ 60  ┆ 78  ┆ 83  ┆ … ┆ 3   ┆ 1   ┆ 1   ┆ 3    │
│ 8    ┆ 28  ┆ 62  ┆ 45  ┆ … ┆ 3   ┆ 6   ┆ 3   ┆ 4    │
│ …    ┆ …   ┆ …   ┆ …   ┆ … ┆ …   ┆ …   ┆ …   ┆ …    │
│ 1    ┆ 6   ┆ 15  ┆ 17  ┆ … ┆ 11  ┆ 3   ┆ 3   ┆ 6    │
│ 0    ┆ 2   ┆ 7   ┆ 5   ┆ … ┆ 8   ┆ 4   ┆ 15  ┆ 7    │
│ 1    ┆ 1   ┆ 3   ┆ 5   ┆ … ┆ 10  ┆ 8   ┆ 3   ┆ 8    │
│ 1    ┆ 0   ┆ 3   ┆ 4   ┆ … ┆ 4   ┆ 5   ┆ 7   ┆ 9    │
│ 0    ┆ 1   ┆ 3   ┆ 0   ┆ … ┆ 12  ┆ 5   ┆ 49  ┆ 10+  │
└──────┴─────┴─────┴─────┴───┴─────┴─────┴─────┴──────┘, _body=<great_tables._gt_data.Body object at 0x11c663340>, _boxhead=Boxhead([ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='0', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='center', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='center', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='center', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10+', column_align='center', column_width=None), ColInfo(var='Year', type=<ColInfoTypeEnum.stub: 2>, column_label='Year', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x138ff3470>, _spanners=Spanners([]), _heading=Heading(title='Joint Distribution of the Purchasing of Alpha in Year 1 & 2', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='0', rows=[0]), grpname=None, colname='0', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#68359a')]), StyleInfo(locname=LocBody(columns='0', rows=[1]), grpname=None, colname='0', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#eae3f1')]), StyleInfo(locname=LocBody(columns='0', rows=[2]), grpname=None, colname='0', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f8f6fb')]), StyleInfo(locname=LocBody(columns='0', rows=[3]), grpname=None, colname='0', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fdfcfe')]), StyleInfo(locname=LocBody(columns='0', rows=[4]), grpname=None, colname='0', rownu